In [ ]:
from datetime import datetime, date, timedelta
import sys
import tkinter as tk
from tkinter import ttk, messagebox
import requests
import browser_cookie3
import pandas as pd
import PublicDataReader as pdr
from datetime import datetime, timedelta
import json
from urllib.parse import quote
import threading
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import matplotlib.font_manager as fm
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
import requests
import io
import os
import time
import random
import webbrowser
import tkinter.filedialog

# 새로 추가된 시계열 다운로드 관련 import
import shutil
import glob
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
from bs4 import BeautifulSoup # 맨 위 import 부분에 추가해 주세요.

# Windows 환경에서만 pywin32 import 및 사용
IS_WINDOWS = os.name == 'nt'
if IS_WINDOWS:
    try:
        import win32gui
        import win32con
        print("pywin32 라이브러리를 성공적으로 로드했습니다.")
    except ImportError:
        print("경고: pywin32 라이브러리가 설치되지 않았습니다. 'pip install pywin32'를 실행하여 설치하세요.")
        IS_WINDOWS = False

# Tkinter 고해상도(High DPI) 지원 시도
if IS_WINDOWS:
    try:
        from ctypes import windll
        windll.shcore.SetProcessDpiAwareness(1)
        print("DPI 인식 설정을 시도했습니다.")
    except:
        print("DPI 인식 설정 중 오류가 발생했거나 지원되지 않는 환경입니다.")
        pass

# 프로그램 만료일 변수
PROGRAM_EXPIRATION_DATE_STR = "2025-12-31"

# 시계열 다운로드 설정 변수
TIMESERIES_DOWNLOAD_DIR = r"C:\Bi시각화\시계열"
KB_BASE_URL = "https://kbland.kr/webview.html#/main/statistics?channel=kbland"
KB_WEEKLY_TAB_XPATH = '//*[@id="__BVID__30___BV_tab_button__"]'
DOWNLOAD_BUTTON_XPATH = '//*[@id="reference2"]/div[1]/button'
INITIAL_LOAD_WAIT = 3
TAB_LOAD_WAIT = 3

class InvestmentTableProgram:
    def __init__(self, program_expiry_date_str):
        # ... (초기화 코드) ...
        # self.create_timeseries_analysis_page(...) 등 UI 생성 메서드 호출
        # 이 시점에 호출되는 모든 메서드는 이 __init__보다 위에 정의되어 있어야 합니다.
        # 즉, create_timeseries_analysis_page 가 여기서 호출되므로,
        # create_timeseries_analysis_page 보다 위에 download_kb_weekly_timeseries 같은
        # 필요한 메서드가 정의되어 있어야 합니다.
        pass # 실제 __init__ 내용은 파도님 코드를 참조

    # ==============================================================================
    # 1. 초기화 및 프로그램 생명주기 관리, 에러 처리, 핵심 유틸리티 메서드
    #    (다른 메서드들이 광범위하게 참조하는 유틸리티성 메서드를 먼저 배치)
    # ==============================================================================
        
        self.root = tk.Tk()
        self.root.title("DAON_Searcher")

        # 프로그램 시작 시 사용 기한 점검
        self.check_usage_expiry(program_expiry_date_str) 

        # Matplotlib 폰트 설정
        try:
            font_name = fm.FontProperties(family='Malgun Gothic').get_name()
            plt.rcParams['font.family'] = font_name
            plt.rcParams['axes.unicode_minus'] = False
        except Exception as e:
            messagebox.showwarning("폰트 설정 오류", f"폰트 설정 중 오류 발생: {e}\n기본 폰트로 대체합니다.")
            plt.rcParams['font.family'] = 'DejaVu Sans'

        self.root.state('zoomed')

        # 페이지 구성 관리
        self.page_order = [1, 2, 3, 4, 5]
        self.page_names = {
            1: "투자테이블", 2: "시세차트", 3: "시계열분석", 4: "페이지4", 5: "페이지5"
        }
        self.current_page = 1 
        self.current_page_frame = None

        # 필터 설정 변수들
        self.region_var = tk.StringVar(value="4481000000") 
        self.sido_var = tk.StringVar()                     
        self.sigungu_var = tk.StringVar()                  
        self.region_data = {}                              

        # 날짜, 면적, 증감율 등 기타 필터 변수
        self.start_date_var = tk.StringVar()
        self.end_date_var = tk.StringVar()
        end_date = datetime.now()
        start_date = end_date - pd.Timedelta(days=180)
        self.start_date_var.set(start_date.strftime("%Y-%m-%d"))
        self.end_date_var.set(end_date.strftime("%Y-%m-%d"))
        self.exclusive_area_min = tk.StringVar(value="0")
        self.exclusive_area_max = tk.StringVar(value="999")
        self.max_years = tk.StringVar(value="30")
        self.max_unit_price = tk.StringVar(value="999999")
        self.rate_min = tk.StringVar(value="-100")
        self.rate_max = tk.StringVar(value="100")

        # 데이터 저장 변수
        self.raw_data = None
        self.filtered_data = None
        self.selected_apartments = {}

        # 시계열 분석 관련 변수 추가
        self.timeseries_download_path_var = tk.StringVar()
        self.set_default_timeseries_download_path()
        self.timeseries_sido_var = tk.StringVar()
        self.timeseries_sigungu_var = tk.StringVar()
        self.timeseries_period_start_var = tk.StringVar()
        self.timeseries_period_end_var = tk.StringVar()
        
        # 차트 데이터 선택 변수들 (새로 추가)
        self.chart_data_vars = {
            '매수심리': tk.BooleanVar(value=True),  # 초기에 선택됨
            '전세현황': tk.BooleanVar(value=False),
            '전세수급': tk.BooleanVar(value=False),
            '매매전망': tk.BooleanVar(value=False),
            '전세전망': tk.BooleanVar(value=False),
            '매매지수': tk.BooleanVar(value=False),
            '전세지수': tk.BooleanVar(value=False),
            '매매증감': tk.BooleanVar(value=False),
            '전세증감': tk.BooleanVar(value=False)
        }
        
        # 기본 기간 설정 (최근 1년)
        end_date_ts = datetime.now()
        start_date_ts = end_date_ts - pd.Timedelta(days=365)
        self.timeseries_period_start_var.set(start_date_ts.strftime("%Y-%m-%d"))
        self.timeseries_period_end_var.set(end_date_ts.strftime("%Y-%m-%d"))
        
        # 시계열 차트 관련 변수
        self.timeseries_chart_frame = None
        self.timeseries_fig = None
        self.timeseries_ax = None
        self.timeseries_canvas = None
        self.current_timeseries_data = None

        # 엑셀 내보내기 경로 저장 변수 추가
        self.export_path_var = tk.StringVar()
        self.set_default_export_path()

        # 차트 관련 변수
        self.fig = None; self.ax = None; self.ax2 = None; self.canvas = None
        self.chart_info_labels = {}; self.current_chart_data = None
        self.current_selected_apt_info = None

        # 차트 옵션 변수
        self.show_sise_sale_var = tk.BooleanVar(value=True)
        self.show_sise_lease_var = tk.BooleanVar(value=True)
        self.show_sise_ratio_var = tk.BooleanVar(value=True)
        self.show_real_sale_var = tk.BooleanVar(value=True)
        self.show_real_lease_var = tk.BooleanVar(value=True)
        self.chart_start_date_var = tk.StringVar()
        self.chart_end_date_var = tk.StringVar()

        # 실거래 리스트용 Treeviews
        self.sale_real_tran_tree = None
        self.lease_real_tran_tree = None

        # 네이버 부동산 매물 검색 관련 변수들
        self.naver_danji_map = {}
        self.naver_myeoneok_map = {}
        self.naver_danji_combobox = None
        self.naver_myeoneok_combobox = None
        self.buy_property_tree = None; self.rent_property_tree = None
        self.current_trade_type = tk.StringVar(value="매매") 
        self.naver_summary_labels = {} 
        self.article_buy_data = None; self.article_rent_data = None
        self.debug_label = None 
        self.buy_property_tree_sort_direction = False
        self.rent_property_tree_sort_direction = False

        # 지역코드.txt 로드 메서드 호출
        self.load_region_data() 

        # UI 레이아웃 핵심
        self.main_frame = ttk.Frame(self.root)
        self.main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # 내비게이션 바
        self.create_navigation_bar(self.main_frame) 

        # 페이지 내용이 표시될 컨텐츠 프레임
        self.content_frame = ttk.Frame(self.main_frame)
        self.content_frame.pack(fill=tk.BOTH, expand=True, pady=(10, 0))

        # 로딩 레이블
        self.loading_label = ttk.Label(self.main_frame, text="프로그램 준비 완료", font=('Arial', 12))
        self.loading_label.pack(side=tk.BOTTOM, fill=tk.X, pady=5)

        # 페이지 프레임 생성 및 UI 그리기 호출
        self.page_frames = {} 
        
        # 1페이지 (투자테이블) 프레임 생성 및 UI 그리기
        self.page_frames[1] = ttk.Frame(self.content_frame)
        self.create_investment_table_page(self.page_frames[1])

        # 2페이지 (시세차트) 프레임 생성 및 UI 그리기
        self.page_frames[2] = ttk.Frame(self.content_frame)
        self.create_chart_page(self.page_frames[2])

        # 3페이지 (시계열분석) 프레임 생성 및 UI 그리기
        self.page_frames[3] = ttk.Frame(self.content_frame)
        self.create_timeseries_analysis_page(self.page_frames[3])

        # 4, 5페이지는 임시 페이지
        self.page_frames[4] = ttk.Frame(self.content_frame)
        ttk.Label(self.page_frames[4], text="4페이지 입니다. (임시)", font=('Arial', 20)).pack(pady=50)
        
        self.page_frames[5] = ttk.Frame(self.content_frame)
        ttk.Label(self.page_frames[5], text="5페이지 입니다. (임시)", font=('Arial', 20)).pack(pady=50)

        # 시계열 다운로드 폴더 생성 확인
        self.ensure_timeseries_download_dir()

        # 프로그램 시작 시 초기 페이지 로드
        self.load_page(1)

    # User-Agent 기본 헤더 설정
    DEFAULT_HEADERS = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Accept": "application/json, text/plain, */*",
        "Accept-Language": "ko-KR,ko;q=0.9,en;q=0.8",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive",
        "Referer": "https://new.land.naver.com/",
        "Origin": "https://new.land.naver.com",
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-site"
    }

    def check_usage_expiry(self, expiry_date_str):
        """프로그램 사용 기한 점검"""
        try:
            expiration_date = datetime.strptime(expiry_date_str, "%Y-%m-%d").date()
        except ValueError:
            messagebox.showwarning("기한 설정 오류", f"제공된 만료일 형식이 올바르지 않습니다: {expiry_date_str}. (YYYY-MM-DD 형식이어야 합니다.)")
            return

        today = date.today()
        days_left = (expiration_date - today).days

        if today > expiration_date:
            messagebox.showwarning(
                "사용 기한 만료",
                f"프로그램 사용 기한({expiration_date})이 만료되었습니다.\n지속적인 사용을 위해 관리자 승인이 필요합니다.",
                parent=self.root
            )
            self.root.destroy() 
            sys.exit() 
            return

        elif 0 <= days_left <= 10:
            messagebox.showwarning(
                "사용 기한 임박",
                f"프로그램 사용 기한이 {days_left}일 남았습니다 ({expiration_date}).\n지속적인 사용을 위해 관리자 승인이 필요합니다.",
                parent=self.root
            )

    def set_default_timeseries_download_path(self):
        """시계열 다운로드 기본 경로 설정"""
        self.timeseries_download_path_var.set(TIMESERIES_DOWNLOAD_DIR)

    def ensure_timeseries_download_dir(self):
        """시계열 다운로드 폴더 생성 확인"""
        download_path = self.timeseries_download_path_var.get()
        if not os.path.exists(download_path):
            try:
                os.makedirs(download_path)
                print(f"시계열 다운로드 폴더를 생성했습니다: {download_path}")
            except OSError as e:
                messagebox.showerror("오류", f"시계열 다운로드 폴더 생성 실패: {e}")

    def show_error(self, error_msg):
        """일반 오류 메시지를 표시하는 범용 메서드"""
        self.loading_label.config(text="오류가 발생했습니다.")
        messagebox.showerror("오류", error_msg)        

    def show_property_search_error(self, error_msg):
        """매물 검색 오류 표시 (show_error를 사용할 수도 있지만 별도 로직이 있다면 여기 배치)"""
        self.loading_label.config(text="매물 검색 실패")
        messagebox.showerror("매물 검색 오류", error_msg)
        # 오류 발생 시 매물 현황 레이블 초기화
        for trade_type_key, display_name in [('buy', '매매'), ('rent', '전세')]:
            self.naver_summary_labels[f'{trade_type_key}_count'].config(text=f"{display_name} 매물건수: -개")
            self.naver_summary_labels[f'{trade_type_key}_max_price'].config(text=f"{display_name} 최고가: -억원")
            self.naver_summary_labels[f'{trade_type_key}_min_price'].config(text=f"{display_name} 최저가: -억원")
        
        # Treeview도 비워주기
        for item in self.buy_property_tree.get_children():
            self.buy_property_tree.delete(item)
        for item in self.rent_property_tree.get_children():
            self.rent_property_tree.delete(item)

    def update_debug_label(self, message):
        """디버깅 라벨 텍스트를 업데이트하는 메서드"""
        if self.debug_label:
           self.debug_label.config(text=message)
           self.root.update_idletasks()

    def fetch_naver_api_data(self, url):
        """
        네이버 부동산 API에서 데이터를 가져오는 범용 메서드
        - 브라우저에서 복사한 쿠키를 하드코딩
        - browser_cookie3 없이 동작
        """
        try:
            headers = {
                'accept': 'application/json, text/plain, */*',
                'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
                'referer': 'https://fin.land.naver.com/',
                'user-agent': (
                    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                    'AppleWebKit/537.36 (KHTML, like Gecko) '
                    'Chrome/138.0.0.0 Safari/537.36'
                ),
                # ★ 여기 쿠키 값은 크롬 개발자도구에서 복사해서 붙여넣으세요
                'cookie': 'NAC=LcV5BogfnITfC; NNB=53JOHN6SM5RGQ; SHOW_FIN_BADGE=Y; bnb_tooltip_shown_finance_v1=true; _fwb=220FhMGmpQ2BtTNICGStK6Z.1752616946035; landHomeFlashUseYn=Y; ASID=7a2b3579000001985a70dc080000001b; nhn.realestate.article.rlet_type_cd=A01; nhn.realestate.article.trade_type_cd=""; realestate.beta.lastclick.cortar=4400000000; PROP_TEST_KEY=1754632854081.db0b658403da2bf0f24b506613cf8d9acf878c7af22296020db8166f2844b2db; PROP_TEST_ID=44995f4422776c30b611ccf580060a39260c013b9d8179a5129d683c639425e5; page_uid=j5kX6dqo1LVssM9bZP8ssssssJ0-281326; NACT=1; SRT30=1755142689; SRT5=1755142689; BUC=G3KGLDoPM5S83hxSi_0iws26uo9I8bUF-RVCvmS3OMo=',
            }

            session = requests.Session()
            session.headers.update(headers)

            # 1~3초 랜덤 대기 (429 방지)
            time.sleep(random.uniform(5, 10))

            response = session.get(url, timeout=10)
            response.raise_for_status()

            if response.headers.get('content-type', '').startswith('application/json'):
                return response.json()
            else:
                print(f"경고: JSON 응답이 아님. Content-Type: {response.headers.get('content-type')}")
                print(response.text[:500])
                return None

        except requests.exceptions.RequestException as e:
            print(f"API 요청 오류: {e}")
            if e.response is not None:
                print(f"응답 코드: {e.response.status_code}")
                print(e.response.text[:500])
            return None
        except Exception as e:
            print(f"예상치 못한 오류 발생: {e}")
            return None
            
    def set_default_export_path(self):
        """Excel 내보내기 기본 경로 설정"""
        home_dir = os.path.expanduser("~")
        download_dir = os.path.join(home_dir, "Downloads")
        self.export_path_var.set(download_dir)

    def load_region_data(self):
        """지역 코드 파일 로드 및 파싱 (콤보박스 등에 필요)"""
        # GitHub Raw URL
        github_raw_url = "https://raw.githubusercontent.com/kaga-fo/DaonSearcher_Project/refs/heads/main/%EC%A7%80%EC%97%AD%EC%BD%94%EB%93%9C.txt"
        
        # 로컬 파일 경로
        local_fallback_path = r"C:\Users\kagaj\code\지역코드.txt"
        
        self.region_data = {}

        file_source_name = ""
        file_content_object = None

        try:
            # 1. GitHub에서 로드 시도
            file_source_name = "GitHub"
            print(f"[{file_source_name}]에서 지역코드 파일 로드 시도: {github_raw_url}")
            response = requests.get(github_raw_url, timeout=10)
            response.raise_for_status()

            file_content_object = io.StringIO(response.text)
            print(f"[{file_source_name}]에서 지역코드 파일 성공적으로 로드.")

        except requests.exceptions.RequestException as e:
            # 2. GitHub 로드 실패 시 로컬 파일 시도
            print(f"[{file_source_name}]에서 지역코드 파일 로드 중 오류 발생: {e}. 로컬 파일로 대체 시도.")
            if os.path.exists(local_fallback_path):
                file_source_name = "로컬 파일"
                try:
                    file_content_object = open(local_fallback_path, 'r', encoding='utf-8-sig') 
                    print(f"[{file_source_name}] '{local_fallback_path}' 성공적으로 로드.")
                except Exception as ex:
                    messagebox.showerror("오류", f"로컬 지역코드 파일 로드 중 오류 발생: {str(ex)}\n파일 인코딩 또는 경로를 확인해주세요.")
                    self.region_data = {}
                    return
            else:
                messagebox.showerror("오류", f"지역코드 파일이 없습니다.\nGitHub에서도 로컬({local_fallback_path})에서도 찾을 수 없습니다.")
                self.region_data = {}
                return
        except Exception as e:
            messagebox.showerror("오류", f"지역코드 파일 처리 중 알 수 없는 오류 발생: {str(e)}")
            self.region_data = {}
            return

        # 파싱 로직
        try:
            for line in file_content_object:
                line = line.strip()
                if not line:
                    continue
                parts = line.split()
                
                if len(parts) >= 3: 
                    code = parts[0]
                    sido = parts[1]
                    sigungu = parts[2]
                    if sido not in self.region_data:
                        self.region_data[sido] = {}
                    self.region_data[sido][sigungu] = code
                elif len(parts) == 2: 
                    code = parts[0]
                    sido = parts[1]
                    if sido not in self.region_data:
                        self.region_data[sido] = {}
                    self.region_data[sido][''] = code
                else:
                    print(f"경고: 지역코드 파일 '{file_source_name}'의 형식이 올바르지 않습니다 (최소 2개 항목 필요): {line}")

            print(f"지역코드 파싱 완료. 총 {len(self.region_data)}개의 시도 데이터.")
        except Exception as e:
            messagebox.showerror("오류", f"지역코드 파일 파싱 중 오류 발생: {str(e)}")
            self.region_data = {}
        finally:
            if hasattr(file_content_object, 'close'): 
                file_content_object.close()

    def wait_for_download_completion(self, download_path, timeout=60):
        """
        [핵심 수정 사항: 이 메서드를 반드시 클래스 안으로 옮겨야 합니다.]
        다운로드 완료 대기 유틸리티 메서드 (아마 _run_kb_weekly/monthly_download에서 호출)
        """
        print(f"다운로드 폴더 '{download_path}'에서 새로운 파일 다운로드를 기다립니다... (최대 {timeout}초)")
        seconds = 0
        last_size = -1
        file_path = None
        start_time = time.time()

        existing_files = {os.path.join(download_path, f): os.path.getctime(os.path.join(download_path, f))
                          for f in os.listdir(download_path) if os.path.isfile(os.path.join(download_path, f))}

        while time.time() - start_time < timeout:
            time.sleep(1)
            seconds += 1

            list_of_files = [os.path.join(download_path, f) for f in os.listdir(download_path) if os.path.isfile(os.path.join(download_path, f))]

            if not list_of_files:
                 continue

            new_files = [(f, os.path.getctime(f)) for f in list_of_files if f not in existing_files]

            if new_files:
                latest_new_file, latest_ctime = max(new_files, key=lambda item: item[1])

                if not latest_new_file.endswith('.crdownload'):
                     file_path = latest_new_file
                     current_size = os.path.getsize(file_path)

                     if current_size > 0 and current_size == last_size and seconds > 2:
                         print(f"파일 크기 안정됨: {current_size} bytes. 다운로드 완료 추정.")
                         return file_path
                     else:
                         last_size = current_size

        print(f"다운로드 완료 대기 시간 초과 ({timeout}초). 새로운 파일이 감지되지 않았거나 다운로드가 완료되지 않았습니다.")
        return None

    def process_downloaded_file(self, downloaded_file_path, target_name):
        """다운로드된 파일을 복사하고 원하는 이름으로 변경하여 저장"""
        if not downloaded_file_path or not os.path.exists(downloaded_file_path):
            print("다운로드된 파일을 찾을 수 없습니다.")
            return False

        download_path = self.timeseries_download_path_var.get()
        target_file_name = f"{target_name}.xlsx"
        target_file_path = os.path.join(download_path, target_file_name)

        print(f"다운로드된 원본 파일: {os.path.basename(downloaded_file_path)}")
        print(f"복사본 대상 파일명: {target_file_name}")

        # 기존에 같은 이름의 대상 파일이 있으면 삭제
        if os.path.exists(target_file_path):
            print(f"기존 대상 파일 삭제 시도: {target_file_name}")
            try:
                os.remove(target_file_path)
                print("기존 파일 삭제 완료.")
            except OSError as e:
                print(f"기존 대상 파일 삭제 실패: {e}")
                messagebox.showerror("오류", f"기존 파일 삭제 실패: {target_file_name}\n{e}\n파일이 사용 중인지 확인해 보세요.")
                return False

        # 다운로드된 파일을 원하는 이름으로 복사
        try:
            shutil.copy2(downloaded_file_path, target_file_path)
            print(f"파일 복사 완료: {os.path.basename(downloaded_file_path)} -> {target_file_name}")
            return True
        except OSError as e:
            print(f"파일 복사 실패: {e}")
            messagebox.showerror("오류", f"파일 복사 실패: {os.path.basename(downloaded_file_path)} -> {target_file_name}\n{e}")
            return False

    # ==============================================================================
    # 2. 특정 기능 그룹 - 다운로드 관련 메서드
    #    (UI 생성 메서드에서 직접 command로 연결되므로 UI 생성보다 위에 배치해야 함)
    # ==============================================================================

    def download_kb_weekly_timeseries(self):
        """KB 주간 시계열 다운로드 (버튼 command)"""
        self.loading_label.config(text="KB 주간 시계열 다운로드를 시작합니다...")
        
        # 스레드에서 실행하여 UI가 멈추지 않도록 함
        thread = threading.Thread(target=self._run_kb_weekly_download)
        thread.daemon = True
        thread.start()

    def download_kb_monthly_timeseries(self):
        """KB 월간 시계열 다운로드 (버튼 command)"""
        self.loading_label.config(text="KB 월간 시계열 다운로드를 시작합니다...")
        
        # 스레드에서 실행하여 UI가 멈추지 않도록 함
        thread = threading.Thread(target=self._run_kb_monthly_download)
        thread.daemon = True
        thread.start()

    def _run_kb_weekly_download(self):
        """KB 주간 시계열 다운로드 실행 (스레드 타겟)"""
        driver = None
        downloaded_file_path = None
        
        try:
            download_path = self.timeseries_download_path_var.get()
            self.ensure_timeseries_download_dir()

            # Chrome 옵션 설정
            chrome_options = webdriver.ChromeOptions()
            prefs = {
                "download.default_directory": download_path,
                "download.prompt_for_download": False,
                "download.directory_upgrade": True,
                "safebrowsing.enabled": True
            }
            chrome_options.add_experimental_option("prefs", prefs)
            chrome_options.add_argument("--headless")
            chrome_options.add_argument("--no-sandbox")
            chrome_options.add_argument("--disable-dev-shm-usage")

            print("KB 주간 시계열 다운로드 시작...")
            driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)

            print(f"페이지 접속: {KB_BASE_URL}")
            driver.get(KB_BASE_URL)

            # 페이지 로딩 및 초기화 대기
            print(f"페이지 로딩 및 초기화 대기 ({INITIAL_LOAD_WAIT}초)...")
            time.sleep(INITIAL_LOAD_WAIT)

            # '주간시계열' 탭 클릭
            try:
                print(f"주간시계열 탭 찾기 및 클릭 시도: {KB_WEEKLY_TAB_XPATH}")
                weekly_tab = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.XPATH, KB_WEEKLY_TAB_XPATH))
                )
                weekly_tab.click()
                print("주간시계열 탭 클릭 완료.")
                print(f"탭 내용 로딩 및 다운로드 버튼 활성화 대기 ({TAB_LOAD_WAIT}초)...")
                time.sleep(TAB_LOAD_WAIT)

            except (NoSuchElementException, TimeoutException) as e:
                error_msg = f"주간시계열 탭을 찾거나 클릭할 수 없습니다.\n{e}"
                self.root.after(0, lambda: messagebox.showerror("오류", error_msg))
                return
            except Exception as e:
                error_msg = f"주간시계열 탭 클릭 중 오류 발생.\n{e}"
                self.root.after(0, lambda: messagebox.showerror("오류", error_msg))
                return

            # 다운로드 버튼 클릭
            try:
                print(f"다운로드 버튼 찾기 및 클릭 시도: {DOWNLOAD_BUTTON_XPATH}")
                download_button = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.XPATH, DOWNLOAD_BUTTON_XPATH))
                )
                download_button.click()
                print("다운로드 버튼 클릭 완료.")
            except (NoSuchElementException, TimeoutException) as e:
                error_msg = f"다운로드 버튼을 찾거나 클릭할 수 없습니다.\n{e}"
                self.root.after(0, lambda: messagebox.showerror("오류", error_msg))
                return
            except Exception as e:
                error_msg = f"다운로드 버튼 클릭 중 오류 발생.\n{e}"
                self.root.after(0, lambda: messagebox.showerror("오류", error_msg))
                return

            # 파일 다운로드 완료 대기
            print("파일 다운로드 완료 대기...")
            downloaded_file_path = self.wait_for_download_completion(download_path, timeout=45)

            if downloaded_file_path:
                print("다운로드된 파일 확인됨. 파일 처리 시작...")
                success = self.process_downloaded_file(downloaded_file_path, "주간시계열")
                if success:
                    self.root.after(0, lambda: messagebox.showinfo("완료", "KB 주간 시계열 다운로드 완료"))
                    self.root.after(0, lambda: self.loading_label.config(text="KB 주간 시계열 다운로드 완료"))
                else:
                    self.root.after(0, lambda: self.loading_label.config(text="파일 처리 실패"))
            else:
                error_msg = "KB 주간 시계열 다운로드 중 시간 초과 또는 파일 감지 실패."
                self.root.after(0, lambda: messagebox.showerror("오류", error_msg))
                self.root.after(0, lambda: self.loading_label.config(text="다운로드 실패"))

        except WebDriverException as e:
            error_msg = f"웹 브라우저 제어 중 오류 발생.\n{e}"
            self.root.after(0, lambda: messagebox.showerror("WebDriver 오류", error_msg))
            self.root.after(0, lambda: self.loading_label.config(text="다운로드 실패"))
        except Exception as e:
            error_msg = f"KB 주간 시계열 다운로드 중 오류 발생: {e}"
            self.root.after(0, lambda: messagebox.showerror("알 수 없는 오류", error_msg))
            self.root.after(0, lambda: self.loading_label.config(text="다운로드 실패"))

        finally:
            if driver:
                driver.quit()
                print("브라우저 종료.")
            print("KB 주간 시계열 다운로드 프로세스 종료.")

    def _run_kb_monthly_download(self):
        """KB 월간 시계열 다운로드 실행 (스레드 타겟)"""
        driver = None
        downloaded_file_path = None

        try:
            download_path = self.timeseries_download_path_var.get()
            self.ensure_timeseries_download_dir()

            # Chrome 옵션 설정
            chrome_options = webdriver.ChromeOptions()
            prefs = {
                "download.default_directory": download_path,
                "download.prompt_for_download": False,
                "download.directory_upgrade": True,
                "safebrowsing.enabled": True
            }
            chrome_options.add_experimental_option("prefs", prefs)
            chrome_options.add_argument("--headless")
            chrome_options.add_argument("--no-sandbox")
            chrome_options.add_argument("--disable-dev-shm-usage")

            print("KB 월간 시계열 다운로드 시작...")
            driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)

            print(f"페이지 접속: {KB_BASE_URL}")
            driver.get(KB_BASE_URL)

            # 페이지 로딩 및 초기화 대기
            print(f"페이지 로딩 및 초기화 대기 ({INITIAL_LOAD_WAIT}초)...")
            time.sleep(INITIAL_LOAD_WAIT)

            # 다운로드 버튼 클릭 (월간시계열은 기본 탭이므로 탭 클릭 불필요)
            try:
                print(f"다운로드 버튼 찾기 및 클릭 시도: {DOWNLOAD_BUTTON_XPATH}")
                download_button = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.XPATH, DOWNLOAD_BUTTON_XPATH))
                )
                download_button.click()
                print("다운로드 버튼 클릭 완료.")

            except (NoSuchElementException, TimeoutException) as e:
                error_msg = f"다운로드 버튼을 찾거나 클릭할 수 없습니다.\n{e}"
                self.root.after(0, lambda: messagebox.showerror("오류", error_msg))
                return
            except Exception as e:
                error_msg = f"다운로드 버튼 클릭 중 오류 발생.\n{e}"
                self.root.after(0, lambda: messagebox.showerror("오류", error_msg))
                return

            # 파일 다운로드 완료 대기
            print("파일 다운로드 완료 대기...")
            downloaded_file_path = self.wait_for_download_completion(download_path, timeout=45)

            if downloaded_file_path:
                print("다운로드된 파일 확인됨. 파일 처리 시작...")
                success = self.process_downloaded_file(downloaded_file_path, "월간시계열")
                if success:
                    self.root.after(0, lambda: messagebox.showinfo("완료", "KB 월간 시계열 다운로드 완료"))
                    self.root.after(0, lambda: self.loading_label.config(text="KB 월간 시계열 다운로드 완료"))
                else:
                    self.root.after(0, lambda: self.loading_label.config(text="파일 처리 실패"))
            else:
                error_msg = "KB 월간 시계열 다운로드 중 시간 초과 또는 파일 감지 실패."
                self.root.after(0, lambda: messagebox.showerror("오류", error_msg))
                self.root.after(0, lambda: self.loading_label.config(text="다운로드 실패"))

        except WebDriverException as e:
            error_msg = f"웹 브라우저 제어 중 오류 발생.\n{e}"
            self.root.after(0, lambda: messagebox.showerror("WebDriver 오류", error_msg))
            self.root.after(0, lambda: self.loading_label.config(text="다운로드 실패"))
        except Exception as e:
            error_msg = f"KB 월간 시계열 다운로드 중 오류 발생: {e}"
            self.root.after(0, lambda: messagebox.showerror("알 수 없는 오류", error_msg))
            self.root.after(0, lambda: self.loading_label.config(text="다운로드 실패"))

        finally:
            if driver:
                driver.quit()
                print("브라우저 종료.")
            print("KB 월간 시계열 다운로드 프로세스 종료.")


    # ==============================================================================
    # 3. UI 생성 메서드들
    #    (__init__에서 호출되며, 앞에서 정의된 유틸리티/다운로드 메서드를 활용)
    # ==============================================================================

    def create_navigation_bar(self, parent):
        """메인 내비게이션 바 생성"""
        nav_frame = ttk.Frame(parent)
        nav_frame.pack(fill=tk.X, pady=(0, 10))

        ttk.Button(nav_frame, text="페이지 순서 변경", command=self.change_page_order).pack(side=tk.LEFT, padx=(0, 20))

        self.page_buttons = {}
        for i, page_num in enumerate(self.page_order):
            btn = ttk.Button(nav_frame, text=f"{i+1}. {self.page_names[page_num]}", command=lambda p=page_num: self.load_page(p))
            btn.pack(side=tk.LEFT, padx=5)
            self.page_buttons[page_num] = btn

    def create_investment_table_page(self, parent_frame):
        """1페이지: 투자테이블 UI 생성"""
        title_label = ttk.Label(parent_frame, text="투자테이블", font=('Arial', 16, 'bold')) 
        title_label.pack(pady=(0, 20))

        filter_frame = ttk.LabelFrame(parent_frame, text="필터 설정", padding=10) 
        filter_frame.pack(fill=tk.X, pady=(0, 10))

        self.create_filter_ui(filter_frame)

        result_frame = ttk.LabelFrame(parent_frame, text="조회 결과", padding=10)
        result_frame.pack(fill=tk.BOTH, expand=True, pady=(10, 0))

        self.create_result_table(result_frame)

    def create_filter_ui(self, parent):
        """투자테이블 필터 UI 생성"""
        row = 0
        padx_val = 3
        pady_val = 3

        # 컬럼 가중치 설정 (8개 열)
        parent.grid_columnconfigure(0, weight=0, minsize=40)
        parent.grid_columnconfigure(1, weight=1)             
        parent.grid_columnconfigure(2, weight=0, minsize=40)
        parent.grid_columnconfigure(3, weight=1)             
        parent.grid_columnconfigure(4, weight=0, minsize=20)
        parent.grid_columnconfigure(5, weight=1)
        parent.grid_columnconfigure(6, weight=1)
        parent.grid_columnconfigure(7, weight=0, minsize=40)

        # Row 0: 시도, 시군구
        ttk.Label(parent, text="시도:").grid(row=row, column=0, sticky=tk.W, padx=padx_val, pady=pady_val)
        self.sido_combobox = ttk.Combobox(parent, textvariable=self.sido_var, values=list(self.region_data.keys()), state="readonly", width=15)
        self.sido_combobox.grid(row=row, column=1, sticky=tk.W, padx=padx_val, pady=pady_val)
        self.sido_combobox.bind("<<ComboboxSelected>>", self.on_sido_selected)

        ttk.Label(parent, text="시군구:").grid(row=row, column=2, sticky=tk.W, padx=padx_val, pady=pady_val)
        self.sigungu_combobox = ttk.Combobox(parent, textvariable=self.sigungu_var, state="readonly", width=15)
        self.sigungu_combobox.grid(row=row, column=3, sticky=tk.W, padx=padx_val, pady=pady_val)

        if self.region_data:
            first_sido = list(self.region_data.keys())[0]
            self.sido_var.set(first_sido)
            self.on_sido_selected(None)
        else:
            self.sido_combobox.config(state="disabled")
            self.sigungu_combobox.config(state="disabled")
            messagebox.showwarning("경고", "지역코드 파일 로드에 실패하여 지역 선택 기능을 사용할 수 없습니다.")
        row += 1 

        # Row 1: 시작일, 종료일
        ttk.Label(parent, text="시작일 (YYYY-MM-DD):").grid(row=row, column=0, sticky=tk.W, padx=padx_val, pady=pady_val)
        ttk.Entry(parent, textvariable=self.start_date_var, width=15).grid(row=row, column=1, sticky=tk.W, padx=padx_val, pady=pady_val)
        ttk.Label(parent, text="종료일 (YYYY-MM-DD):").grid(row=row, column=2, sticky=tk.W, padx=padx_val, pady=pady_val)
        ttk.Entry(parent, textvariable=self.end_date_var, width=15).grid(row=row, column=3, sticky=tk.W, padx=padx_val, pady=pady_val)
        row += 1 

        # Row 2: 전용면적 (최소/최대)
        ttk.Label(parent, text="전용면적 (최소):").grid(row=row, column=0, sticky=tk.W, padx=padx_val, pady=pady_val)
        ttk.Entry(parent, textvariable=self.exclusive_area_min, width=15).grid(row=row, column=1, sticky=tk.W, padx=padx_val, pady=pady_val)
        ttk.Label(parent, text="전용면적 (최대):").grid(row=row, column=2, sticky=tk.W, padx=padx_val, pady=pady_val)
        ttk.Entry(parent, textvariable=self.exclusive_area_max, width=15).grid(row=row, column=3, sticky=tk.W, padx=padx_val, pady=pady_val)
        row += 1 

        # Row 3: 연차, 평단가
        ttk.Label(parent, text="연차 (최대):").grid(row=row, column=0, sticky=tk.W, padx=padx_val, pady=pady_val)
        ttk.Entry(parent, textvariable=self.max_years, width=15).grid(row=row, column=1, sticky=tk.W, padx=padx_val, pady=pady_val)
        ttk.Label(parent, text="최대 평단가:").grid(row=row, column=2, sticky=tk.W, padx=padx_val, pady=pady_val)
        ttk.Entry(parent, textvariable=self.max_unit_price, width=15).grid(row=row, column=3, sticky=tk.W, padx=padx_val, pady=pady_val)
        row += 1 

        # Row 4: 증감율 (최소/최대)
        ttk.Label(parent, text="증감율 (최소%):").grid(row=row, column=0, sticky=tk.W, padx=padx_val, pady=pady_val)
        ttk.Entry(parent, textvariable=self.rate_min, width=15).grid(row=row, column=1, sticky=tk.W, padx=padx_val, pady=pady_val)
        ttk.Label(parent, text="증감율 (최대%):").grid(row=row, column=2, sticky=tk.W, padx=padx_val, pady=pady_val)
        ttk.Entry(parent, textvariable=self.rate_max, width=15).grid(row=row, column=3, sticky=tk.W, padx=padx_val, pady=pady_val)
        row += 1

        # Row 5: 모든 버튼과 경로 UI 배치
        # 조회 버튼
        ttk.Button(parent, text="조회", command=self.search_data).grid(row=row, column=0, columnspan=2, pady=10)
    
        # Excel 내보내기 버튼
        ttk.Button(parent, text="Excel 내보내기", command=self.export_to_excel).grid(row=row, column=2, columnspan=2, pady=10)

        # 저장 경로 라벨
        ttk.Label(parent, text="저장 경로:", font=('Arial', 9)).grid(row=row, column=4, sticky=tk.W, padx=padx_val, pady=pady_val)
    
        # 저장 경로 표시 라벨
        self.path_display_label = ttk.Label(parent, textvariable=self.export_path_var, wraplength=180, justify=tk.LEFT, font=('Arial', 9))
        self.path_display_label.grid(row=row, column=5, columnspan=2, sticky="ew", padx=padx_val, pady=pady_val)
    
        # 경로 변경 버튼
        ttk.Button(parent, text="변경", command=self.choose_export_path, width=5).grid(row=row, column=7, sticky=tk.E, padx=padx_val, pady=pady_val)


    def create_result_table(self, parent):
        """투자테이블 결과 Treeview 생성"""
        table_frame = ttk.Frame(parent)
        table_frame.pack(fill=tk.BOTH, expand=True)

        columns = ('지역', '단지명', '전용면적', '연차', '평단가', '매매증감율', '전세증감율', '매매시세', '전세시세', '전세가율', '매전갭', '세대수')
        self.tree = ttk.Treeview(table_frame, columns=columns, show='tree headings', height=20)

        # === 색상 태그 설정 ===
        # 매매증감율 관련 색상
        self.tree.tag_configure('positive_sale', background='#E8F5E8', foreground='#2E7D32')  # 연한 녹색 배경, 진한 녹색 글자
        self.tree.tag_configure('negative_sale', background='#FFEBEE', foreground='#C62828')  # 연한 빨간색 배경, 진한 빨간색 글자
        
        # 전세증감율 관련 색상
        self.tree.tag_configure('positive_lease', background='#E3F2FD', foreground='#1565C0')  # 연한 파란색 배경, 진한 파란색 글자
        self.tree.tag_configure('negative_lease', background='#FFF3E0', foreground='#E65100')  # 연한 주황색 배경, 진한 주황색 글자
        
        # 전세가율 관련 색상
        self.tree.tag_configure('high_ratio', background='#FCE4EC', foreground='#AD1457')     # 높은 전세가율 (90% 이상)
        
        # 기본 zebra striping
        self.tree.tag_configure('oddrow', background='#F8F8F8')
        self.tree.tag_configure('evenrow', background='#FFFFFF')
        
        # 중성 색상
        self.tree.tag_configure('neutral', background='#F5F5F5', foreground='#424242')

        self.tree.column("#0", width=30, minwidth=30, stretch=tk.NO)
        self.tree.heading("#0", text="", anchor=tk.CENTER)

        column_widths = {
            '지역': 120, '단지명': 150, '전용면적': 80, '연차': 60, '평단가': 100,
            '매매증감율': 100, '전세증감율': 100, '매매시세': 100, '전세시세': 100,
            '전세가율': 80, '매전갭': 80, '세대수': 80
        }

        self.sort_directions = {}

        # === 컬럼 헤더 스타일링 ===
        style = ttk.Style()
        
        # 기본 헤더 스타일
        style.configure("Treeview.Heading", 
                       background="#2196F3",   # 파란색 배경
                       foreground="black",     # 흰색 글자
                       font=("Arial", 10, "bold"))
        
        # 마우스 오버 효과
        style.map("Treeview.Heading",
                  background=[('active', '#1976D2')])  # 마우스 오버시 더 진한 파란색

        for col in columns:
            self.tree.heading(col, text=col, command=lambda _col=col: self._sort_column(_col))
            self.tree.column(col, width=column_widths.get(col, 100), anchor=tk.CENTER)
            self.sort_directions[col] = False

        scrollbar_y = ttk.Scrollbar(table_frame, orient=tk.VERTICAL, command=self.tree.yview)
        scrollbar_x = ttk.Scrollbar(table_frame, orient=tk.HORIZONTAL, command=self.tree.xview)

        self.tree.configure(yscrollcommand=scrollbar_y.set, xscrollcommand=scrollbar_x.set)

        self.tree.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        scrollbar_y.pack(side=tk.RIGHT, fill=tk.Y)
        scrollbar_x.pack(side=tk.BOTTOM, fill=tk.X)

        self.tree.bind("<<TreeviewSelect>>", self._on_tree_select)

    def create_chart_page(self, parent_frame):
        """2페이지: 시세차트 UI 생성"""
        pane_window = ttk.PanedWindow(parent_frame, orient=tk.HORIZONTAL)
        pane_window.pack(fill=tk.BOTH, expand=True)
        
        left_frame = ttk.Frame(pane_window, width=200, relief=tk.SUNKEN)
        pane_window.add(left_frame, weight=1)

        ttk.Label(left_frame, text="선택 단지 목록", font=('Arial', 14, 'bold')).pack(pady=10)

        selected_apt_columns = ('지역', '단지명', '전용면적', '연차')
        self.selected_apt_tree = ttk.Treeview(left_frame, columns=selected_apt_columns, show='headings', height=15)

        for col in selected_apt_columns:
            self.selected_apt_tree.heading(col, text=col, anchor=tk.CENTER)
            self.selected_apt_tree.column(col, width=80, anchor=tk.CENTER)

        selected_apt_scrollbar_y = ttk.Scrollbar(left_frame, orient=tk.VERTICAL, command=self.selected_apt_tree.yview)
        self.selected_apt_tree.configure(yscrollcommand=selected_apt_scrollbar_y.set)

        self.selected_apt_tree.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=5, pady=5)
        selected_apt_scrollbar_y.pack(side=tk.RIGHT, fill=tk.Y)

        for item_id, apt_info in self.selected_apartments.items():
            self.selected_apt_tree.insert('', tk.END, values=(
                apt_info.get('지역', ''), apt_info.get('단지명', ''),
                apt_info.get('전용면적', ''), apt_info.get('연차', '')
            ), iid=item_id)

        self.selected_apt_tree.bind("<<TreeviewSelect>>", self.on_selected_apt_tree_select)

        # 중앙 프레임
        center_frame = ttk.Frame(pane_window, relief=tk.SUNKEN)
        pane_window.add(center_frame, weight=6) # weight를 5에서 6으로 변경하여 차트 공간 확대

        center_frame.grid_rowconfigure(0, weight=0)  # 옵션창 고정 높이
        center_frame.grid_rowconfigure(1, weight=6)  # 시세차트 (높이 비율 3에 해당)
        center_frame.grid_rowconfigure(2, weight=2)  # 거래건수 차트 (높이 비율 1에 해당)
        center_frame.grid_columnconfigure(0, weight=1)

        # 차트 옵션창
        chart_options_frame = ttk.LabelFrame(center_frame, text="차트 옵션", padding=10)
        chart_options_frame.grid(row=0, column=0, sticky="ew", padx=5, pady=5)

        # 차트 기간 설정 UI
        ttk.Label(chart_options_frame, text="차트 시작일:").grid(row=0, column=0, padx=5, pady=2, sticky=tk.W)
        ttk.Entry(chart_options_frame, textvariable=self.chart_start_date_var, width=15).grid(row=0, column=1, padx=5, pady=2, sticky=tk.W)
        ttk.Label(chart_options_frame, text="차트 종료일:").grid(row=0, column=2, padx=5, pady=2, sticky=tk.W)
        ttk.Entry(chart_options_frame, textvariable=self.chart_end_date_var, width=15).grid(row=0, column=3, padx=5, pady=2, sticky=tk.W)
        ttk.Button(chart_options_frame, text="기간 적용", command=self.update_chart_view).grid(row=0, column=4, padx=10, pady=2)

        # 차트 표시 유형 선택 체크버튼
        ttk.Checkbutton(chart_options_frame, text="매매가", variable=self.show_sise_sale_var, command=self.update_chart_view).grid(row=1, column=0, padx=5, pady=2, sticky=tk.W)
        ttk.Checkbutton(chart_options_frame, text="전세가", variable=self.show_sise_lease_var, command=self.update_chart_view).grid(row=1, column=1, padx=5, pady=2, sticky=tk.W)
        ttk.Checkbutton(chart_options_frame, text="전세가율", variable=self.show_sise_ratio_var, command=self.update_chart_view).grid(row=1, column=2, padx=5, pady=2, sticky=tk.W)
        ttk.Checkbutton(chart_options_frame, text="매매실거래", variable=self.show_real_sale_var, command=self.update_chart_view).grid(row=1, column=3, padx=5, pady=2, sticky=tk.W)
        ttk.Checkbutton(chart_options_frame, text="전세실거래", variable=self.show_real_lease_var, command=self.update_chart_view).grid(row=1, column=4, padx=5, pady=2, sticky=tk.W)

        # 시세차트 영역 (row=1)
        self.chart_frame = ttk.Frame(center_frame, relief=tk.GROOVE, borderwidth=2)
        self.chart_frame.grid(row=1, column=0, sticky="nsew", padx=5, pady=5)

        self.fig = Figure(figsize=(6, 6), dpi=100)
        gs = self.fig.add_gridspec(nrows=2, ncols=1, height_ratios=[3, 1])  # 3:1 비율 설정
        self.ax = self.fig.add_subplot(gs[0, 0])  # 상단 시세차트
        self.ax2 = self.fig.add_subplot(gs[1, 0], sharex=self.ax)  # 하단 거래 건수 차트, x축 공유

        self.canvas = FigureCanvasTkAgg(self.fig, master=self.chart_frame)
        self.canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)

        # 차트 하단 주요 가격 정보 (row=2)
        self.chart_info_frame = ttk.Frame(center_frame, relief=tk.GROOVE, borderwidth=2)
        self.chart_info_frame.grid(row=2, column=0, sticky="nsew", padx=5, pady=5)

        # 3분할 프레임 및 레이블 생성
        price_info_frame1 = ttk.Frame(self.chart_info_frame)
        price_info_frame1.grid(row=0, column=0, sticky="nsew", padx=5, pady=5)
        price_info_frame2 = ttk.Frame(self.chart_info_frame)
        price_info_frame2.grid(row=0, column=1, sticky="nsew", padx=5, pady=5)
        price_info_frame3 = ttk.Frame(self.chart_info_frame)
        price_info_frame3.grid(row=0, column=2, sticky="nsew", padx=5, pady=5)
        
        self.chart_info_frame.grid_columnconfigure(0, weight=1)
        self.chart_info_frame.grid_columnconfigure(1, weight=1)
        self.chart_info_frame.grid_columnconfigure(2, weight=1)

        # 좌측 섹션: 매매 최고/최근가, 전세 최고/최근가
        ttk.Label(price_info_frame1, text="<시세 정보>", font=('Arial', 10, 'bold')).pack(anchor=tk.W, pady=(0, 5))
        self.chart_info_labels['매매최고가'] = ttk.Label(price_info_frame1, text="매매최고가: -")
        self.chart_info_labels['매매최고가'].pack(anchor=tk.W)
        self.chart_info_labels['매매최근가'] = ttk.Label(price_info_frame1, text="매매최근가: -")
        self.chart_info_labels['매매최근가'].pack(anchor=tk.W)
        self.chart_info_labels['전세최고가'] = ttk.Label(price_info_frame1, text="전세최고가: -")
        self.chart_info_labels['전세최고가'].pack(anchor=tk.W)
        self.chart_info_labels['전세최근가'] = ttk.Label(price_info_frame1, text="전세최근가: -")
        self.chart_info_labels['전세최근가'].pack(anchor=tk.W)

        # 가운데 섹션: 실거래 매매/전세 최고/최근가
        ttk.Label(price_info_frame2, text="<실거래 정보>", font=('Arial', 10, 'bold')).pack(anchor=tk.W, pady=(0, 5))
        self.chart_info_labels['실거래매매최고가'] = ttk.Label(price_info_frame2, text="실거래매매최고가: -")
        self.chart_info_labels['실거래매매최고가'].pack(anchor=tk.W)
        self.chart_info_labels['실거래매매최근가'] = ttk.Label(price_info_frame2, text="실거래매매최근가: -")
        self.chart_info_labels['실거래매매최근가'].pack(anchor=tk.W)
        self.chart_info_labels['실거래전세최고가'] = ttk.Label(price_info_frame2, text="실거래전세최고가: -")
        self.chart_info_labels['실거래전세최고가'].pack(anchor=tk.W)
        self.chart_info_labels['실거래전세최근가'] = ttk.Label(price_info_frame2, text="실거래전세최근가: -")
        self.chart_info_labels['실거래전세최근가'].pack(anchor=tk.W)

        # 우측 섹션: 매매 하락가/율, 회복율
        ttk.Label(price_info_frame3, text="<매매 분석>", font=('Arial', 10, 'bold')).pack(anchor=tk.W, pady=(0, 5))
        self.chart_info_labels['매매하락가'] = ttk.Label(price_info_frame3, text="매매하락가: -")
        self.chart_info_labels['매매하락가'].pack(anchor=tk.W)
        self.chart_info_labels['매매하락율'] = ttk.Label(price_info_frame3, text="매매하락율: -")
        self.chart_info_labels['매매하락율'].pack(anchor=tk.W)
        self.chart_info_labels['매매회복율'] = ttk.Label(price_info_frame3, text="매매회복율: -")
        self.chart_info_labels['매매회복율'].pack(anchor=tk.W)

        # 우측 프레임
        right_frame = ttk.PanedWindow(pane_window, orient=tk.VERTICAL)
        pane_window.add(right_frame, weight=1) # weight를 2에서 1로 변경하여 우측 공간 축소

        # 우측 상단 - 실거래 정보창 (좌우 2분할)
        top_right_frame = ttk.Frame(right_frame, relief=tk.GROOVE, borderwidth=2)
        right_frame.add(top_right_frame, weight=1)  # weight를 2로 변경 (40%)

        top_right_frame.grid_columnconfigure(0, weight=1)
        top_right_frame.grid_columnconfigure(1, weight=1)
        top_right_frame.grid_rowconfigure(0, weight=0)  # 라벨 행 고정
        top_right_frame.grid_rowconfigure(1, weight=1)  # Treeview 행 확장

        # 매매 실거래 Treeview (좌측 컬럼)
        sale_label_frame = ttk.Frame(top_right_frame)
        sale_label_frame.grid(row=0, column=0, sticky="ew", padx=2, pady=2)
        ttk.Label(sale_label_frame, text="매매 실거래", font=('Arial', 10, 'bold')).pack(pady=(5, 0))

        sale_tree_frame = ttk.Frame(top_right_frame)
        sale_tree_frame.grid(row=1, column=0, sticky="nsew", padx=2, pady=2)
        sale_real_tran_cols = ('날짜', '실거래금액', '층수')
        self.sale_real_tran_tree = ttk.Treeview(sale_tree_frame, columns=sale_real_tran_cols, show='headings')  # height 제거
        for col in sale_real_tran_cols:
            self.sale_real_tran_tree.heading(col, text=col, anchor=tk.CENTER)
            self.sale_real_tran_tree.column(col, width=70, anchor=tk.CENTER)
        self.sale_real_tran_tree.pack(fill=tk.BOTH, expand=True)  # expand=True로 부모 프레임 채움

        # 전세/월세 실거래 Treeview (우측 컬럼)
        lease_label_frame = ttk.Frame(top_right_frame)
        lease_label_frame.grid(row=0, column=1, sticky="ew", padx=2, pady=2)
        ttk.Label(lease_label_frame, text="전세/월세 실거래", font=('Arial', 10, 'bold')).pack(pady=(5, 0))

        lease_tree_frame = ttk.Frame(top_right_frame)
        lease_tree_frame.grid(row=1, column=1, sticky="nsew", padx=2, pady=2)
        lease_real_tran_cols = ('날짜', '실거래금액', '층수')
        self.lease_real_tran_tree = ttk.Treeview(lease_tree_frame, columns=lease_real_tran_cols, show='headings')  # height 제거
        for col in lease_real_tran_cols:
            self.lease_real_tran_tree.heading(col, text=col, anchor=tk.CENTER)
            self.lease_real_tran_tree.column(col, width=70, anchor=tk.CENTER)
        self.lease_real_tran_tree.pack(fill=tk.BOTH, expand=True)  # expand=True로 부모 프레임 채움

        # 우측 하단 - 네이버 부동산 매물 검색창
        bottom_right_frame = ttk.Frame(right_frame, relief=tk.GROOVE, borderwidth=2)
        right_frame.add(bottom_right_frame, weight=4)  # weight를 3으로 변경 (60%)
        
        self.create_naver_property_search(bottom_right_frame)

    def create_timeseries_analysis_page(self, parent_frame):
        """
        3페이지: 시계열 분석 UI 생성
        [핵심: 이 메서드보다 download_kb_weekly_timeseries 등이 위에 정의되어야 합니다.]
        """
        # 메인 PanedWindow (3분할: 좌측, 중앙, 우측)
        main_paned = ttk.PanedWindow(parent_frame, orient=tk.HORIZONTAL)
        main_paned.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)

        # === 좌측 영역 (다운로드 및 설정) ===
        left_frame = ttk.Frame(main_paned, width=300)
        main_paned.add(left_frame, weight=1)

        # 좌측 상단: 다운로드 버튼 및 경로 설정
        download_frame = ttk.LabelFrame(left_frame, text="시계열 다운로드", padding=10)
        download_frame.pack(fill=tk.X, padx=5, pady=5)

        # 다운로드 버튼들
        btn_frame = ttk.Frame(download_frame)
        btn_frame.pack(fill=tk.X, pady=(0, 10))

        kb_weekly_btn = ttk.Button(btn_frame, text="KB 주간시계열", 
                                  command=self.download_kb_weekly_timeseries)
        kb_weekly_btn.pack(side=tk.LEFT, padx=(0, 5), fill=tk.X, expand=True)

        kb_monthly_btn = ttk.Button(btn_frame, text="KB 월간시계열", 
                                   command=self.download_kb_monthly_timeseries)
        kb_monthly_btn.pack(side=tk.LEFT, padx=(5, 0), fill=tk.X, expand=True)

        # 다운로드 경로 설정
        path_frame = ttk.Frame(download_frame)
        path_frame.pack(fill=tk.X, pady=(0, 5))

        ttk.Label(path_frame, text="다운로드 경로:").pack(anchor=tk.W)
        
        path_entry_frame = ttk.Frame(path_frame)
        path_entry_frame.pack(fill=tk.X, pady=(5, 0))
        
        path_entry = ttk.Entry(path_entry_frame, textvariable=self.timeseries_download_path_var)
        path_entry.pack(side=tk.LEFT, fill=tk.X, expand=True, padx=(0, 5))
        
        path_browse_btn = ttk.Button(path_entry_frame, text="찾기", 
                                    command=self.browse_timeseries_download_path)
        path_browse_btn.pack(side=tk.RIGHT)

        # 좌측 하단: 분석 기준 선택
        analysis_frame = ttk.LabelFrame(left_frame, text="분석 설정", padding=10)
        analysis_frame.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)

        # 시도/시군구 선택 (1행 2열)
        region_frame = ttk.Frame(analysis_frame)
        region_frame.pack(fill=tk.X, pady=(0, 10))

        # 1열: 시도 선택
        sido_frame = ttk.Frame(region_frame)
        sido_frame.pack(side=tk.LEFT, fill=tk.X, expand=True, padx=(0, 5))
        ttk.Label(sido_frame, text="시도:").pack(anchor=tk.W)
        self.timeseries_sido_combobox = ttk.Combobox(sido_frame, textvariable=self.timeseries_sido_var, 
                                                    state="readonly")
        self.timeseries_sido_combobox.pack(fill=tk.X, pady=(2, 0))
        self.timeseries_sido_combobox.bind("<<ComboboxSelected>>", self.on_timeseries_sido_selected)

        # 2열: 시군구 선택
        sigungu_frame = ttk.Frame(region_frame)
        sigungu_frame.pack(side=tk.LEFT, fill=tk.X, expand=True, padx=(5, 0))
        ttk.Label(sigungu_frame, text="시군구:").pack(anchor=tk.W)
        self.timeseries_sigungu_combobox = ttk.Combobox(sigungu_frame, textvariable=self.timeseries_sigungu_var, 
                                                       state="readonly")
        self.timeseries_sigungu_combobox.pack(fill=tk.X, pady=(2, 0))

        # 3열: 기간 설정
        period_frame = ttk.Frame(analysis_frame)
        period_frame.pack(fill=tk.X, pady=(0, 10))

        ttk.Label(period_frame, text="분석 기간:").pack(anchor=tk.W)
        
        period_input_frame = ttk.Frame(period_frame)
        period_input_frame.pack(fill=tk.X, pady=(2, 0))

        ttk.Label(period_input_frame, text="시작:").pack(side=tk.LEFT)
        start_entry = ttk.Entry(period_input_frame, textvariable=self.timeseries_period_start_var, width=12)
        start_entry.pack(side=tk.LEFT, padx=(5, 10))

        ttk.Label(period_input_frame, text="종료:").pack(side=tk.LEFT)
        end_entry = ttk.Entry(period_input_frame, textvariable=self.timeseries_period_end_var, width=12)
        end_entry.pack(side=tk.LEFT, padx=(5, 0))

        # === 차트 데이터 선택 영역 (새로 추가) ===
        chart_data_frame = ttk.LabelFrame(analysis_frame, text="차트 데이터 선택", padding=10)
        chart_data_frame.pack(fill=tk.X, pady=(10, 0))

        # 체크버튼들을 3x3 그리드로 배치
        chart_options = [
            ('매수심리', '매수심리'),
            ('전세현황', '전세현황'),
            ('전세수급', '전세수급'),
            ('매매전망', '매매전망'),
            ('전세전망', '전세전망'),
            ('매매지수', '매매지수'),
            ('전세지수', '전세지수'),
            ('매매증감', '매매증감'),
            ('전세증감', '전세증감')
        ]

        for i, (key, display_text) in enumerate(chart_options):
            row = i // 3
            col = i % 3
            cb = ttk.Checkbutton(chart_data_frame, text=display_text, 
                                variable=self.chart_data_vars[key],
                                command=self.on_chart_data_selection_changed)
            cb.grid(row=row, column=col, sticky=tk.W, padx=5, pady=2)

        # 전체 선택/해제 버튼
        select_all_frame = ttk.Frame(chart_data_frame)
        select_all_frame.grid(row=3, column=0, columnspan=3, pady=(10, 0))

        ttk.Button(select_all_frame, text="전체 선택", 
                  command=self.select_all_chart_data).pack(side=tk.LEFT, padx=(0, 5))
        ttk.Button(select_all_frame, text="전체 해제", 
                  command=self.deselect_all_chart_data).pack(side=tk.LEFT)

        # 분석 실행 버튼
        analyze_btn = ttk.Button(analysis_frame, text="차트 생성", 
                               command=self.generate_timeseries_chart)
        analyze_btn.pack(fill=tk.X, pady=(10, 0))

        # === 우측 영역 (차트 표시) ===
        right_frame = ttk.Frame(main_paned)
        main_paned.add(right_frame, weight=2)

        # 차트 프레임
        self.timeseries_chart_frame = ttk.LabelFrame(right_frame, text="시계열 분석 차트", padding=10)
        self.timeseries_chart_frame.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)

        # 초기 차트 설정
        self.setup_timeseries_chart()

        # 시도/시군구 콤보박스 초기 데이터 로드
        self.load_timeseries_region_data()

    def create_naver_property_search(self, parent):
        """네이버 부동산 매물 검색 UI 생성"""
        # ttk.Label(parent, text="네이버 부동산 매물 검색", font=('Arial', 12, 'bold')).pack(pady=(5, 10)) 
        
        # 검색 조건 프레임
        search_frame = ttk.LabelFrame(parent, text="검색 조건", padding=5)
        search_frame.pack(fill=tk.X, padx=5, pady=5)
        
        # --- 삭제됨 (기존 첫 번째 줄: 시도, 시군구, 법정동 UI 요소) ---
        # row1_frame = ttk.Frame(search_frame)
        # row1_frame.pack(fill=tk.X, pady=2)
        
        # ttk.Label(row1_frame, text="시도:").pack(side=tk.LEFT, padx=(0, 5))
        # self.naver_sido_combobox = ttk.Combobox(row1_frame, state="readonly", width=8)
        # self.naver_sido_combobox.pack(side=tk.LEFT, padx=(0, 10))
        # self.naver_sido_combobox.bind("<<ComboboxSelected>>", self.on_naver_sido_selected)
        
        # ttk.Label(row1_frame, text="시군구:").pack(side=tk.LEFT, padx=(0, 5))
        # self.naver_sigungu_combobox = ttk.Combobox(row1_frame, state="readonly", width=12)
        # self.naver_sigungu_combobox.pack(side=tk.LEFT)
        # self.naver_sigungu_combobox.bind("<<ComboboxSelected>>", self.on_naver_sigungu_selected)
        
        # # 법정동 (row1에 포함)
        # ttk.Label(row1_frame, text="법정동:").pack(side=tk.LEFT, padx=(0, 5))
        # self.naver_bjdong_combobox = ttk.Combobox(row1_frame, state="readonly", width=8)
        # self.naver_bjdong_combobox.pack(side=tk.LEFT, padx=(0, 10))
        # self.naver_bjdong_combobox.bind("<<ComboboxSelected>>", self.on_naver_bjdong_selected)
        # --- 삭제됨 끝 ---
        
        # --- 수정 시작 (단지 콤보박스 배치 조정 - 이제 첫 번째 라인이 됨) ---
        danji_frame = ttk.Frame(search_frame)
        danji_frame.pack(fill=tk.X, pady=2)
        
        ttk.Label(danji_frame, text="단지:").pack(side=tk.LEFT, padx=(0, 5))
        self.naver_danji_combobox = ttk.Combobox(danji_frame, state="readonly", width=45)
        self.naver_danji_combobox.pack(side=tk.LEFT)
        # self.naver_danji_combobox.bind("<<ComboboxSelected>>", self.on_naver_danji_selected) # 이 바인딩은 삭제됨
        # --- 수정 끝 ---
        
        # --- 수정 시작 (면적 콤보박스 및 검색 버튼 배치 조정 - 이제 두 번째 라인이 됨) ---
        myeoneok_search_frame = ttk.Frame(search_frame)
        myeoneok_search_frame.pack(fill=tk.X, pady=2)
        
        ttk.Label(myeoneok_search_frame, text="면적:").pack(side=tk.LEFT, padx=(0, 5))
        self.naver_myeoneok_combobox = ttk.Combobox(myeoneok_search_frame, state="readonly", width=30)
        self.naver_myeoneok_combobox.pack(side=tk.LEFT, padx=(0, 10))
        self.naver_myeoneok_combobox.bind("<<ComboboxSelected>>", self.on_naver_myeoneok_selected) 
        
        ttk.Button(myeoneok_search_frame, text="매물 검색", command=self.search_naver_properties).pack(side=tk.LEFT, padx=(10, 0))
        # --- 수정 끝 ---
               
        # --- 매물 현황 섹션부터는 기존과 동일하게 진행됩니다 ---
        summary_frame = ttk.LabelFrame(parent, text="매물 현황", padding=5)
        summary_frame.pack(fill=tk.X, padx=5, pady=5)
        
        # 각 레이블들을 딕셔너리에 저장하여 접근하기 쉽게 함
        self.naver_summary_labels['buy_count'] = ttk.Label(summary_frame, text="매매 매물건수: -개")
        self.naver_summary_labels['buy_max_price'] = ttk.Label(summary_frame, text="매매 최고가: -억원")
        self.naver_summary_labels['buy_min_price'] = ttk.Label(summary_frame, text="매매 최저가: -억원")

        self.naver_summary_labels['rent_count'] = ttk.Label(summary_frame, text="전세 매물건수: -개")
        self.naver_summary_labels['rent_max_price'] = ttk.Label(summary_frame, text="전세 최고가: -억원")
        self.naver_summary_labels['rent_min_price'] = ttk.Label(summary_frame, text="전세 최저가: -억원")

        # 레이블 배치 (그리드 사용)
        self.naver_summary_labels['buy_count'].grid(row=0, column=0, sticky=tk.W, padx=5, pady=1)
        self.naver_summary_labels['buy_max_price'].grid(row=1, column=0, sticky=tk.W, padx=5, pady=1)
        self.naver_summary_labels['buy_min_price'].grid(row=2, column=0, sticky=tk.W, padx=5, pady=1)

        summary_frame.grid_columnconfigure(0, weight=1) # 매매 열 확장
        self.naver_summary_labels['rent_count'].grid(row=0, column=1, sticky=tk.W, padx=5, pady=1)
        self.naver_summary_labels['rent_max_price'].grid(row=1, column=1, sticky=tk.W, padx=5, pady=1)
        self.naver_summary_labels['rent_min_price'].grid(row=2, column=1, sticky=tk.W, padx=5, pady=1)

        summary_frame.grid_columnconfigure(1, weight=1) # 전세 열 확장
        # --- 매물 현황 섹션 끝 ---

        # --- 매물 목록 프레임 (매매/전세 분할) ---
        property_list_pane = ttk.PanedWindow(parent, orient=tk.HORIZONTAL)
        property_list_pane.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        property_columns = ('가격', '동', '층', '면적')

        # 매매 매물 목록 Treeview
        buy_list_frame = ttk.LabelFrame(property_list_pane, text="매매 매물", padding=5)
        property_list_pane.add(buy_list_frame, weight=1)
        
        self.buy_property_tree = ttk.Treeview(buy_list_frame, columns=property_columns, show='headings', height=10)
        for col in property_columns:
            self.buy_property_tree.heading(col, text=col, anchor=tk.CENTER)
            if col == '가격':
                self.buy_property_tree.heading(col, text=col, command=lambda c=col: self._sort_naver_property_tree(self.buy_property_tree, c, '매매'))
                self.buy_property_tree.column(col, width=80, anchor=tk.E) 
            elif col == '동':
                self.buy_property_tree.column(col, width=50, anchor=tk.CENTER) 
            elif col == '층':
                self.buy_property_tree.column(col, width=40, anchor=tk.CENTER) 
            elif col == '면적':
                self.buy_property_tree.column(col, width=50, anchor=tk.CENTER) 
        
        # 스크롤바 (화면에는 숨김)
        buy_scrollbar = ttk.Scrollbar(buy_list_frame, orient=tk.VERTICAL, command=self.buy_property_tree.yview)
        self.buy_property_tree.configure(yscrollcommand=buy_scrollbar.set)
        
        self.buy_property_tree.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        self.buy_property_tree.bind("<Double-1>", self._open_naver_article_webpage) # 함수 바인딩

        # 전세 매물 목록 Treeview
        rent_list_frame = ttk.LabelFrame(property_list_pane, text="전세 매물", padding=5)
        property_list_pane.add(rent_list_frame, weight=1)

        self.rent_property_tree = ttk.Treeview(rent_list_frame, columns=property_columns, show='headings', height=10)
        for col in property_columns:
            self.rent_property_tree.heading(col, text=col, anchor=tk.CENTER)
            if col == '가격':
                self.rent_property_tree.heading(col, text=col, command=lambda c=col: self._sort_naver_property_tree(self.rent_property_tree, c, '전세'))
                self.rent_property_tree.column(col, width=80, anchor=tk.E) 
            elif col == '동':
                self.rent_property_tree.column(col, width=50, anchor=tk.CENTER) 
            elif col == '층':
                self.rent_property_tree.column(col, width=40, anchor=tk.CENTER) 
            elif col == '면적':
                self.rent_property_tree.column(col, width=50, anchor=tk.CENTER) 
        
        # 스크롤바 (화면에는 숨김)
        rent_scrollbar = ttk.Scrollbar(rent_list_frame, orient=tk.VERTICAL, command=self.rent_property_tree.yview)
        self.rent_property_tree.configure(yscrollcommand=rent_scrollbar.set)
        
        self.rent_property_tree.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        self.rent_property_tree.bind("<Double-1>", self._open_naver_article_webpage) # 함수 바인딩
        # --- 매물 목록 프레임 끝 ---
        
        # --- 삭제됨 (초기 데이터 로드) ---
        # self.root.after_idle(self.load_naver_sido_list)
        # --- 삭제됨 끝 ---

        # --- 수정 시작 (debug_label 정의를 다시 추가) ---
        self.debug_label = ttk.Label(parent, text="디버깅 정보가 여기에 표시됩니다.", font=('Arial', 8))
        self.debug_label.pack(pady=5)
        # --- 수정 끝 ---


    # ==============================================================================
    # 4. 데이터 처리 및 특정 UI 요소 업데이트 메서드
    #    (UI 이벤트 핸들러, 스레드 타겟 등)
    # ==============================================================================

    def on_sido_selected(self, event):
        """투자테이블 시도 선택 이벤트 핸들러"""
        selected_sido = self.sido_var.get()
        if selected_sido and selected_sido in self.region_data:
            sigungu_list = list(self.region_data[selected_sido].keys())
            self.sigungu_combobox['values'] = sigungu_list
            if sigungu_list:
                self.sigungu_var.set(sigungu_list[0])
            else:
                self.sigungu_var.set("")
        else:
            self.sigungu_combobox['values'] = []
            self.sigungu_var.set("")

    def _on_tree_select(self, event):
        """투자테이블 Treeview 항목 선택 이벤트 핸들러"""
        selected_items_ids = self.tree.selection()
        temp_selected_apartments = {}
        for item_id in selected_items_ids:
            original_item_found = None
            for original_parsed_item in self.filtered_data:
                if str(original_parsed_item.get('면적일련번호')) == item_id:
                    original_item_found = original_parsed_item
                    break
            if original_item_found:
                temp_selected_apartments[item_id] = {
                    '지역': original_parsed_item.get('지역', ''),
                    '단지명': original_parsed_item.get('단지명', ''),
                    '전용면적': original_parsed_item.get('전용면적', ''),
                    '연차': original_parsed_item.get('연차', ''),
                    '면적일련번호': original_parsed_item.get('면적일련번호', ''),
                    '단지기본일련번호': original_parsed_item.get('단지기본일련번호', '')
                }
        self.selected_apartments = temp_selected_apartments
        if selected_items_ids:
            self_selected_item_info = self.selected_apartments.get(selected_items_ids[-1])
            self.current_selected_apt_info = self_selected_item_info
            
            # --- 삭제 시작 (1페이지에서 네이버 단지 코드 및 면적 로드 로직) ---
            # if selected_item_info and selected_item_info.get('단지기본일련번호'):
            #     complex_serial_no = selected_item_info['단지기본일련번호']
            #     self.loading_label.config(text=f"KB 단지 정보 및 네이버 단지 코드를 조회 중입니다...")
            #     # 스레드를 사용하여 UI가 멈추지 않도록 비동기 처리
            #     threading.Thread(target=self.fetch_naver_complex_code_and_update_ui, 
            #                      args=(complex_serial_no,)).start()
            # else:
            #     self.loading_label.config(text="선택된 단지의 KB 정보가 불완전합니다.")
            #     # 단지 선택이 없거나 불완전할 경우 네이버 검색 필드를 초기화합니다.
            #     self.root.after(0, lambda: self.clear_naver_lower_combos(['danji', 'myeoneok']))
            # --- 삭제 끝 ---

        else:
            self.current_selected_apt_info = None
            # --- 수정 시작 (선택된 항목이 없을 경우 네이버 검색 필드 초기화 - 유지) ---
            self.root.after(0, lambda: self.clear_naver_lower_combos(['danji', 'myeoneok']))
            # --- 수정 끝 ---

    def _sort_column(self, col):
        """투자테이블 Treeview 컬럼 정렬"""
        current_data = []
        for item_id in self.tree.get_children():
            current_data.append((self.tree.set(item_id, col), item_id))

        def get_sort_key(item_tuple):
            value = item_tuple[0]
            try:
                numeric_cols = ['전용면적', '연차', '평단가', '매매증감율', '전세증감율', '매매시세', '전세시세', '전세가율', '매전갭', '세대수']
                if col in numeric_cols:
                    return float(value) if str(value).strip() else -float('inf')
                else:
                    return value
            except ValueError:
                return value

        reverse_sort = not self.sort_directions[col]
        self.sort_directions[col] = reverse_sort
        current_data.sort(key=get_sort_key, reverse=reverse_sort)
        for index, (value, item_id) in enumerate(current_data):
            self.tree.move(item_id, '', index)
        for c in self.tree['columns']:
            if c == col:
                self.tree.heading(c, text=f"{c} {'▼' if reverse_sort else '▲'}")
            else:
                self.tree.heading(c, text=c)

    def fetch_naver_complex_code_and_update_ui(self, complex_serial_no):
        """
        KB 단지기본일련번호를 통해 네이버 단지 코드를 가져오고, 
        네이버 매물 검색 UI의 단지/면적 콤보박스를 업데이트합니다.
        이 메서드는 스레드에서 호출됩니다.
        """
        try:
            # KB API URL 구성
            base_url = "https://api.kbland.kr/land-complex/complex/typInfo"
            params = {"단지기본일련번호": complex_serial_no}
            url = base_url + "?" + "&".join([f"{k}={quote(str(v))}" for k, v in params.items()])

            self.root.after(0, lambda: self.loading_label.config(text=f"KB 단지 상세 정보 API 호출 중: {complex_serial_no}..."))
            
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
                'Accept': 'application/json, text/plain, */*',
                'Accept-Language': 'ko-KR,ko;q=0.9,en;q=0.8',
                'Accept-Encoding': "gzip, deflate, br",
                'Connection': 'keep-alive',
                'Referer': 'https://kbland.kr/',
                'Origin': 'https://kbland.kr'
            }
            session = requests.Session()
            session.headers.update(headers)
            response = session.get(url, timeout=10)
            response.raise_for_status()
            
            data = response.json()

            # API 응답 파싱하여 네이버 단지 코드 추출
            naver_complex_code = None
            naver_complex_name_from_api = None

            if data and data.get('dataBody') and data['dataBody'].get('data'):
                typ_info_list = data['dataBody']['data']
                if typ_info_list and len(typ_info_list) > 0:
                    naver_complex_code = typ_info_list[0].get('네이버단지코드')
                    naver_complex_name_from_api = typ_info_list[0].get('단지명')
            
            if naver_complex_code:
                # --- 수정 시작 (호출하는 메서드 이름 변경) ---
                self.root.after(0, self._update_naver_danji_and_myeoneok_combobox, naver_complex_code, naver_complex_name_from_api)
                # --- 수정 끝 ---
                self.root.after(0, lambda: self.loading_label.config(text="네이버 단지 코드 및 면적 목록 로드 완료."))
            else:
                self.root.after(0, lambda: self.loading_label.config(text="네이버 단지 코드를 찾을 수 없습니다."))
                self.root.after(0, lambda: self.clear_naver_lower_combos(['danji', 'myeoneok']))

        except requests.exceptions.RequestException as e:
            error_msg = f"KB 단지 상세 정보 API 오류: {e}"
            self.root.after(0, self.show_error, error_msg)
            self.root.after(0, lambda: self.loading_label.config(text="KB 단지 정보 로드 실패."))
            self.root.after(0, lambda: self.clear_naver_lower_combos(['danji', 'myeoneok']))
        except Exception as e:
            error_msg = f"네이버 단지 코드 로드 중 알 수 없는 오류: {e}"
            self.root.after(0, self.show_error, error_msg)
            self.root.after(0, lambda: self.loading_label.config(text="네이버 단지 정보 로드 실패."))
            self.root.after(0, lambda: self.clear_naver_lower_combos(['danji', 'myeoneok']))

    def _update_naver_danji_and_myeoneok_combobox(self, naver_danji_code, naver_complex_name_from_api=None):
        """
        네이버 단지 코드(hscpNo)를 바탕으로 네이버 매물 검색의 단지 콤보박스를 설정하고
        해당 단지의 면적 목록을 로드합니다. (Tkinter 메인 스레드에서 호출되어야 함)
        """
        try:
            # 먼저 콤보박스를 초기화합니다.
            self.clear_naver_lower_combos(['danji', 'myeoneok'])
            
            display_text = ""
            # API에서 직접 단지명을 가져온 경우 사용
            if naver_complex_name_from_api:
                display_text = naver_complex_name_from_api
            else:
                # 단지명이 없을 경우 코드로 대체 (임시 방안, 실제로는 단지명을 가져오는 API를 추가하거나 해야 함)
                display_text = f"단지코드: {naver_danji_code}" 

            # naver_danji_map 업데이트: 선택된 하나의 단지만 포함
            self.naver_danji_map = {display_text: naver_danji_code}
            
            # 단지 콤보박스의 values 설정
            self.naver_danji_combobox['values'] = [display_text]
            
            # set() 하기 전에 이벤트 바인딩 해제는 더 이상 필요 없음 (bind 삭제됨)
            self.naver_danji_combobox.set(display_text)
            
            # --- 수정 시작 (단지 콤보박스 상태를 'disabled'로 변경) ---
            self.naver_danji_combobox.config(state="disabled") # 사용자가 변경할 수 없도록
            # --- 수정 끝 ---

            self.root.update_idletasks() # UI 갱신 강제

            # 설정된 단지에 대한 면적 목록 로드
            # --- 수정 시작 (on_naver_danji_selected 대신 load_naver_myeoneok_list 직접 호출) ---
            self.load_naver_myeoneok_list(naver_danji_code)
            # --- 수정 끝 ---

        except Exception as e:
            error_msg = f"네이버 단지 및 면적 콤보박스 업데이트 중 오류 발생: {e}"
            # --- 수정 시작 (에러 메시지 출력) ---
            self.show_error(error_msg)
            # --- 수정 끝 ---
            self.clear_naver_lower_combos(['danji', 'myeoneok'])
    
    def search_data(self):
        """조회 버튼 클릭 시 데이터 검색 시작 (fetch_data 스레드 호출)"""
        selected_sido = self.sido_var.get()
        selected_sigungu = self.sigungu_var.get()

        if not self.region_data:
            messagebox.showwarning("경고", "지역코드 파일이 로드되지 않아 지역 선택 기능을 사용할 수 없습니다.")
            self.loading_label.config(text="조회 실패: 지역 데이터 없음")
            return

        if selected_sido and selected_sido in self.region_data:
            if selected_sigungu in self.region_data[selected_sido]:
                self.region_var.set(self.region_data[selected_sido][selected_sigungu])
            elif '' in self.region_data[selected_sido] and not selected_sigungu:
                self.region_var.set(self.region_data[selected_sido][''])
            else:
                messagebox.showwarning("경고", "유효한 시군구를 선택해주세요.")
                self.loading_label.config(text="조회 실패: 유효한 지역 선택 필요")
                return
        else:
            messagebox.showwarning("경고", "유효한 시도를 선택해주세요.")
            self.loading_label.config(text="조회 실패: 유효한 지역 선택 필요")
            return

        try:
            start_date_obj = datetime.strptime(self.start_date_var.get(), "%Y-%m-%d")
            end_date_obj = datetime.strptime(self.end_date_var.get(), "%Y-%m-%d")
            if start_date_obj > end_date_obj:
                messagebox.showwarning("경고", "시작일은 종료일보다 빨라야 합니다.")
                self.loading_label.config(text="조회 실패: 날짜 범위 오류")
                return
        except ValueError:
            messagebox.showwarning("경고", "날짜 형식이 올바르지 않습니다 (YYYY-MM-DD).")
            self.loading_label.config(text="조회 실패: 날짜 형식 오류")
            return

        self.loading_label.config(text="데이터를 조회 중입니다...")
        thread = threading.Thread(target=self.fetch_data)
        thread.daemon = True
        thread.start()


    def choose_export_path(self):
        """Excel 내보내기 경로 선택"""
        folder_selected = tkinter.filedialog.askdirectory(initialdir=self.export_path_var.get())
        if folder_selected:
            self.export_path_var.set(folder_selected)


    def fetch_data(self):
        """KB API에서 투자 데이터 가져오기 (update_table 호출)"""
        try:
            start_date_str = self.start_date_var.get()
            end_date_str = self.end_date_var.get()

            base_url = "https://api.kbland.kr/land-extra/price/v1/api/invstTblAptSearch"
            params = {
                "시작년월일": start_date_str.replace("-", ""),
                "종료년월일": end_date_str.replace("-", ""),
                "지역리스트": self.region_var.get()
            }
            url = base_url + "?" + "&".join([f"{k}={quote(str(v))}" for k, v in params.items()])

            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
                'Accept': 'application/json, text/plain, */*',
                'Accept-Language': 'ko-KR,ko;q=0.9,en;q=0.8',
                'Accept-Encoding': 'gzip, deflate, br',
                'Connection': 'keep-alive',
                'Referer': 'https://kbland.kr/',
                'Origin': 'https://kbland.kr'
            }
            session = requests.Session()
            session.headers.update(headers)

            max_retries = 3
            for attempt in range(max_retries):
                try:
                    response = session.get(url, timeout=30)
                    response.raise_for_status()
                    break
                except requests.exceptions.ConnectionError as e:
                    if attempt == max_retries - 1:
                        raise e
                    time.sleep(2)

            if response.headers.get('content-type', '').startswith('application/json'):
                data = response.json()
            else:
                raise ValueError(f"JSON 응답이 아닙니다. Content-Type: {response.headers.get('content-type')}")

            self.root.after(0, self.update_table, data)

        except requests.exceptions.ConnectionError as e:
            error_msg = f"연결 오류: API 서버에 연결할 수 없습니다. 네트워크 상태를 확인하거나 잠시 후 다시 시도해주세요."
            self.root.after(0, self.show_error, error_msg)
        except requests.exceptions.Timeout as e:
            error_msg = f"시간 초과: 요청이 시간 초과되었습니다. 잠시 후 다시 시도해주세요."
            self.root.after(0, self.show_error, error_msg)
        except requests.exceptions.HTTPError as e:
            error_msg = f"HTTP 오류: {e.response.status_code} - {e.response.reason}"
            self.root.after(0, self.show_error, error_msg)
        except ValueError as e:
            error_msg = f"데이터 형식 오류: {str(e)}"
            self.root.after(0, self.show_error, error_msg)
        except Exception as e:
            error_msg = f"알 수 없는 오류: {str(e)}"
            self.root.after(0, self.show_error, error_msg)

    def parse_item_data(self, item):
        """KB 투자 데이터 파싱"""
        try:
            region = item.get('지역명', '')
            data_info = item.get('데이터정보', {})

            complex_name = data_info.get('I2010', {}).get('단지명', '')
            area_info = data_info.get('I2020', {})
            exclusive_area = area_info.get('전용면적', '')
            household_count = data_info.get('I2030', {}).get('지수', '')
            years = data_info.get('I2040', {}).get('지수', '')

            unit_price_info = data_info.get('I2050', {})
            unit_price = unit_price_info.get('공급면적당지수', '')
            if isinstance(unit_price, str):
                unit_price = unit_price.replace(',', '')

            sale_info = data_info.get('I2060', {})
            sale_price = sale_info.get('지수', '')
            sale_rate = sale_info.get('증감', '')
            lease_info = data_info.get('I2070', {})
            lease_price = lease_info.get('지수', '')
            lease_rate = lease_info.get('증감', '')
            lease_ratio_info = data_info.get('I2080', {})
            lease_ratio = lease_ratio_info.get('지수', '')

            gap_info = data_info.get('I2090', {})
            gap_value = gap_info.get('지수', '')

            area_serial_no = item.get('면적일련번호', '')
            complex_serial_no = item.get('단지기본일련번호', '')

            return {
                '지역': region, '단지명': complex_name, '전용면적': exclusive_area, '연차': years,
                '평단가': unit_price, '매매증감율': sale_rate, '전세증감율': lease_rate,
                '매매시세': sale_price, '전세시세': lease_price, '전세가율': lease_ratio,
                '매전갭': gap_value, '세대수': household_count,
                '면적일련번호': area_serial_no, '단지기본일련번호': complex_serial_no
            }
        except Exception as e:
            print(f"데이터 파싱 중 오류 발생: {e}, 원본 아이템: {item}")
            return None

    def update_table(self, data):
        """투자테이블 Treeview 업데이트"""
        for item in self.tree.get_children():
            self.tree.delete(item)

        self.raw_data = data
        parsed_data = []

        items_to_parse = []
        if isinstance(data, dict) and 'dataBody' in data and \
           isinstance(data['dataBody'], dict) and 'data' in data['dataBody'] and \
           isinstance(data['dataBody']['data'], dict) and '데이터목록' in data['dataBody']['data']:
            items_to_parse = data['dataBody']['data']['데이터목록']

        for item in items_to_parse:
            parsed_item = self.parse_item_data(item)
            if parsed_item:
                parsed_data.append(parsed_item)

        self.filtered_data = self.filter_data(parsed_data)

        for i, item in enumerate(self.filtered_data):
            values = (
                item.get('지역', ''), item.get('단지명', ''), item.get('전용면적', ''),
                item.get('연차', ''), item.get('평단가', ''), item.get('매매증감율', ''),
                item.get('전세증감율', ''), item.get('매매시세', ''), item.get('전세시세', ''),
                item.get('전세가율', ''), item.get('매전갭', ''), item.get('세대수', '')
            )
            
            # === 색상 로직 적용 ===
            tags = []
            
            # 기본 zebra striping
            base_tag = 'evenrow' if i % 2 == 0 else 'oddrow'
            
            # 매매증감율과 전세증감율을 모두 고려한 우선순위 색상 적용
            sale_rate = None
            lease_rate = None
            lease_ratio = None
            
            try:
                # 매매증감율 파싱
                sale_rate_str = str(item.get('매매증감율', '0')).strip()
                if sale_rate_str and sale_rate_str not in ['', '-', '0']:
                    sale_rate = float(sale_rate_str)
                
                # 전세증감율 파싱
                lease_rate_str = str(item.get('전세증감율', '0')).strip()
                if lease_rate_str and lease_rate_str not in ['', '-', '0']:
                    lease_rate = float(lease_rate_str)
                
                # 전세가율 파싱
                lease_ratio_str = str(item.get('전세가율', '0')).strip()
                if lease_ratio_str and lease_ratio_str not in ['', '-', '0']:
                    lease_ratio = float(lease_ratio_str)
                    
            except (ValueError, TypeError):
                pass
            
            # 색상 우선순위 적용
            color_applied = False
            
            # 1순위: 전세가율이 90% 이상인 경우 (가장 중요한 지표)
            if lease_ratio is not None and lease_ratio >= 90:
                tags = ['high_ratio']
                color_applied = True
            
            # 2순위: 매매증감율이 매우 높거나 낮은 경우
            elif sale_rate is not None:
                if sale_rate >= 10:  # 매매증감율 10% 이상 상승
                    tags = ['positive_sale']
                    color_applied = True
                elif sale_rate <= -10:  # 매매증감율 10% 이상 하락
                    tags = ['negative_sale']
                    color_applied = True
            
            # 3순위: 전세증감율이 매우 높거나 낮은 경우
            if not color_applied and lease_rate is not None:
                if lease_rate >= 10:  # 전세증감율 10% 이상 상승
                    tags = ['positive_lease']
                    color_applied = True
                elif lease_rate <= -10:  # 전세증감율 10% 이상 하락
                    tags = ['negative_lease']
                    color_applied = True
            
            # 4순위: 일반적인 매매증감율 색상
            if not color_applied and sale_rate is not None:
                if sale_rate > 0:
                    tags = ['positive_sale']
                elif sale_rate < 0:
                    tags = ['negative_sale']
                else:
                    tags = [base_tag]  # 0%는 기본 색상
            
            # 색상이 적용되지 않은 경우 기본 zebra striping 적용
            if not tags:
                tags = [base_tag]
            
            # Treeview에 항목 삽입
            self.tree.insert('', tk.END, values=values, iid=str(item.get('면적일련번호', '')), tags=tags)

        self.loading_label.config(text=f"총 {len(self.filtered_data)}개의 결과를 조회했습니다.")
        if not self.filtered_data:
            self.loading_label.config(text="조회 조건에 맞는 데이터가 없습니다.")

    def filter_data(self, data):
        """조회된 데이터 필터링"""
        filtered = []
        for item in data:
            try:
                area_str = str(item.get('전용면적', '0')).strip()
                if area_str and area_str != '':
                    area = float(area_str)
                    if area < float(self.exclusive_area_min.get()) or area > float(self.exclusive_area_max.get()):
                        continue

                years_str = str(item.get('연차', '0')).strip()
                if years_str and years_str != '':
                    years = int(float(years_str))
                    if years > int(self.max_years.get()):
                        continue

                unit_price_str = str(item.get('평단가', '0')).strip()
                if unit_price_str and unit_price_str != '':
                    unit_price = float(unit_price_str)
                    if unit_price > float(self.max_unit_price.get()):
                        continue

                sale_rate_str = str(item.get('매매증감율', '0')).strip()
                if sale_rate_str and sale_rate_str != '':
                    sale_rate = float(sale_rate_str)
                    if sale_rate < float(self.rate_min.get()) or sale_rate > float(self.rate_max.get()):
                        continue
                filtered.append(item)
            except (ValueError, TypeError) as e:
                print(f"필터링 중 오류 발생: {e}, 데이터: {item}")
                filtered.append(item)
                continue
        return filtered

    def export_to_excel(self):
        """Excel로 데이터 내보내기"""
        if not hasattr(self, 'filtered_data') or not self.filtered_data:
            messagebox.showwarning("경고", "내보낼 데이터가 없습니다. 먼저 조회를 실행하세요.")
            return
        try:
            df = pd.DataFrame(self.filtered_data)
            columns_order = ['지역', '단지명', '전용면적', '연차', '평단가', '매매증감율', '전세증감율',
                             '매매시세', '전세시세', '전세가율', '매전갭', '세대수', '면적일련번호', '단지기본일련번호']
            df = df.reindex(columns=columns_order, fill_value='')

            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"투자테이블_{timestamp}.xlsx"

            save_directory = self.export_path_var.get()
            if not save_directory:
                self.set_default_export_path()
                save_directory = self.export_path_var.get()
            
            full_path = os.path.join(save_directory, filename)

            df.to_excel(full_path, index=False, engine='openpyxl')

            messagebox.showinfo("성공", f"데이터가 '{full_path}' 파일로 내보내기 완료되었습니다.")
        except Exception as e:
            messagebox.showerror("오류", f"Excel 내보내기 중 오류가 발생했습니다: {str(e)}")

    def show_error(self, error_msg):
        self.loading_label.config(text="오류가 발생했습니다.")
        messagebox.showerror("오류", error_msg)       

    def change_page_order(self):
        """페이지 순서 변경 다이얼로그 표시"""
        dialog = tk.Toplevel(self.root)
        dialog.title("페이지 순서 변경")
        dialog.geometry("400x300")
        dialog.transient(self.root)
        dialog.grab_set()

        ttk.Label(dialog, text="페이지 순서를 변경하세요 (드래그 앤 드롭 또는 버튼 사용)").pack(pady=10)

        listbox = tk.Listbox(dialog, height=10)
        listbox.pack(fill=tk.BOTH, expand=True, padx=20, pady=10)

        for i, page_num in enumerate(self.page_order):
            listbox.insert(tk.END, f"{i+1}. {self.page_names[page_num]}")

        btn_frame = ttk.Frame(dialog)
        btn_frame.pack(fill=tk.X, padx=20, pady=10)

        def move_up():
            selection = listbox.curselection()
            if selection and selection[0] > 0:
                idx = selection[0]
                self.page_order[idx], self.page_order[idx-1] = self.page_order[idx-1], self.page_order[idx]
                listbox.delete(0, tk.END)
                for i, page_num in enumerate(self.page_order):
                    listbox.insert(tk.END, f"{i+1}. {self.page_names[page_num]}")
                listbox.selection_set(idx-1)

        def move_down():
            selection = listbox.curselection()
            if selection and selection[0] < len(self.page_order) - 1:
                idx = selection[0]
                self.page_order[idx], self.page_order[idx+1] = self.page_order[idx+1], self.page_order[idx]
                listbox.delete(0, tk.END)
                for i, page_num in enumerate(self.page_order):
                    listbox.insert(tk.END, f"{i+1}. {self.page_names[page_num]}")
                listbox.selection_set(idx+1)

        def apply_changes():
            dialog.destroy()
            self.update_navigation_bar()

        ttk.Button(btn_frame, text="위로", command=move_up).pack(side=tk.LEFT, padx=5)
        ttk.Button(btn_frame, text="아래로", command=move_down).pack(side=tk.LEFT, padx=5)
        ttk.Button(btn_frame, text="적용", command=apply_changes).pack(side=tk.RIGHT, padx=5)
        ttk.Button(btn_frame, text="취소", command=dialog.destroy).pack(side=tk.RIGHT, padx=5)

    def update_navigation_bar(self):
        """네비게이션 바 버튼 업데이트"""
        for btn in self.page_buttons.values():
            btn.destroy()

        if self.page_buttons:
            nav_frame = self.page_buttons[list(self.page_buttons.keys())[0]].master
        else:
            nav_frame_parent = self.root.winfo_children()[0]
            nav_frame = None
            for child in nav_frame_parent.winfo_children():
                if isinstance(child, ttk.Frame) and child.winfo_children():
                    nav_frame = child
                    break
            if not nav_frame:
                nav_frame = nav_frame_parent

        self.page_buttons = {}

        for i, page_num in enumerate(self.page_order):
            btn = ttk.Button(nav_frame, text=f"{i+1}. {self.page_names[page_num]}", command=lambda p=page_num: self.load_page(p))
            btn.pack(side=tk.LEFT, padx=5)
            self.page_buttons[page_num] = btn

    def load_page(self, page_num):
        """페이지 전환 및 로드"""
        # 현재 화면에 보이는 페이지 프레임이 있다면 숨김
        if self.current_page_frame:
            self.current_page_frame.pack_forget()

        # 현재 페이지 번호 업데이트
        self.current_page = page_num 

        # 새로 보여줄 페이지 프레임을 가져옴
        new_page_frame = self.page_frames.get(page_num)
        
        if new_page_frame:
            new_page_frame.pack(fill=tk.BOTH, expand=True)
            self.current_page_frame = new_page_frame

            # 2페이지(시세차트)의 경우 차트 캔버스 및 선택 단지 목록 갱신
            if page_num == 2:
                self.root.update_idletasks() 
                self.root.update() 
                self._update_selected_apt_tree_ui()
        else:
            print(f"오류: 페이지 {page_num}에 해당하는 프레임을 찾을 수 없습니다.")

    def on_timeseries_sido_selected(self, event):
        """시계열 분석 시도 선택 이벤트 핸들러"""
        selected_sido = self.timeseries_sido_var.get()
        if selected_sido and selected_sido in self.region_data:
            sigungu_list = list(self.region_data[selected_sido].keys())
            self.timeseries_sigungu_combobox['values'] = sigungu_list
            if sigungu_list:
                self.timeseries_sigungu_var.set(sigungu_list[0])
            else:
                self.timeseries_sigungu_var.set("")
        else:
            self.timeseries_sigungu_combobox['values'] = []
            self.timeseries_sigungu_var.set("")

    def browse_timeseries_download_path(self):
        """시계열 다운로드 경로 탐색"""
        folder_selected = tkinter.filedialog.askdirectory(initialdir=self.timeseries_download_path_var.get())
        if folder_selected:
            self.timeseries_download_path_var.set(folder_selected)

    def generate_timeseries_chart(self):
        """시계열 차트 생성 (plot_multiple_timeseries_data 호출)"""
        # 입력 값 검증
        selected_sido = self.timeseries_sido_var.get()
        selected_sigungu = self.timeseries_sigungu_var.get()
        
        if not selected_sido or not selected_sigungu:
            messagebox.showwarning("경고", "시도와 시군구를 선택해주세요.")
            return

        try:
            start_date = datetime.strptime(self.timeseries_period_start_var.get(), "%Y-%m-%d")
            end_date = datetime.strptime(self.timeseries_period_end_var.get(), "%Y-%m-%d")
            if start_date >= end_date:
                messagebox.showwarning("경고", "시작일은 종료일보다 빨라야 합니다.")
                return
        except ValueError:
            messagebox.showwarning("경고", "날짜 형식이 올바르지 않습니다 (YYYY-MM-DD).")
            return

        # 선택된 차트 데이터 확인
        selected_charts = [key for key, var in self.chart_data_vars.items() if var.get()]
        if not selected_charts:
            messagebox.showwarning("경고", "최소 하나의 차트 데이터를 선택해주세요.")
            return

        # 다운로드 경로에서 시계열 파일 찾기
        download_path = self.timeseries_download_path_var.get()
        weekly_file = os.path.join(download_path, "주간시계열.xlsx")
        monthly_file = os.path.join(download_path, "월간시계열.xlsx")

        # 파일 존재 확인
        available_files = []
        if os.path.exists(weekly_file):
            available_files.append(("주간", weekly_file))
        if os.path.exists(monthly_file):
            available_files.append(("월간", monthly_file))

        if not available_files:
            messagebox.showwarning("경고", "시계열 파일을 찾을 수 없습니다.\n먼저 시계열 데이터를 다운로드해주세요.")
            return

        # 데이터 로드 및 차트 생성
        self.loading_label.config(text="시계열 데이터를 분석하고 차트를 생성 중입니다...")
        
        try:
            self.plot_multiple_timeseries_data(available_files, selected_sido, selected_sigungu, 
                                             start_date, end_date, selected_charts)
            self.loading_label.config(text="시계열 차트가 성공적으로 생성되었습니다.")
        except Exception as e:
            messagebox.showerror("오류", f"시계열 차트 생성 중 오류가 발생했습니다:\n{str(e)}")
            self.loading_label.config(text="시계열 차트 생성에 실패했습니다.")

    def on_chart_data_selection_changed(self):
        """차트 데이터 선택 변경 이벤트 핸들러"""
        # 선택된 항목이 있는지 확인
        selected_count = sum(var.get() for var in self.chart_data_vars.values())
        
        if selected_count == 0:
            # 모든 선택이 해제된 경우 매수심리를 다시 선택
            self.chart_data_vars['매수심리'].set(True)
            messagebox.showinfo("안내", "최소 하나의 차트 데이터를 선택해야 합니다. 매수심리가 자동으로 선택되었습니다.")

    def select_all_chart_data(self):
        """모든 차트 데이터 선택"""
        for var in self.chart_data_vars.values():
            var.set(True)

    def deselect_all_chart_data(self):
        """모든 차트 데이터 선택 해제"""
        for key, var in self.chart_data_vars.items():
            if key == '매수심리':
                var.set(True)  # 매수심리는 항상 선택 상태 유지
            else:
                var.set(False)

    def setup_timeseries_chart(self):
        """시계열 차트 초기 설정"""
        self.timeseries_fig = Figure(figsize=(10, 8), dpi=100)
        self.timeseries_ax = self.timeseries_fig.add_subplot(111)
        self.timeseries_ax.set_title("시계열 분석 차트")
        self.timeseries_ax.set_xlabel("날짜")
        self.timeseries_ax.set_ylabel("지수/값")
        self.timeseries_ax.text(0.5, 0.5, "지역을 선택하고 차트 생성 버튼을 클릭하세요", 
                               ha='center', va='center', transform=self.timeseries_ax.transAxes,
                               fontsize=12)

        self.timeseries_canvas = FigureCanvasTkAgg(self.timeseries_fig, master=self.timeseries_chart_frame)
        self.timeseries_canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

    def load_timeseries_region_data(self):
        """시계열 분석용 시도/시군구 데이터 로드"""
        if self.region_data:
            sido_names = list(self.region_data.keys())
            self.timeseries_sido_combobox['values'] = sido_names
            if sido_names:
                self.timeseries_sido_var.set(sido_names[0])
                self.on_timeseries_sido_selected(None)

    def plot_multiple_timeseries_data(self, available_files, sido, sigungu, start_date, end_date, selected_charts):
        """다중 시계열 데이터 플롯"""
        self.timeseries_ax.clear()
        
        # 차트 데이터별 색상 및 스타일 정의
        chart_styles = {
            '매수심리': {'매도자많음': {'color': 'blue', 'style': '-'}, '매수자많음': {'color': 'red', 'style': '-'}},
            '전세현황': {'수요>공급': {'color': 'blue', 'style': '-'}, '수요<공급': {'color': 'red', 'style': '-'}},
            '전세수급': {'전세수급지수': {'color': 'gold', 'style': '-'}},
            '매매전망': {'전망차이': {'color': 'purple', 'style': '-'}},
            '전세전망': {'전망차이': {'color': 'orange', 'style': '-'}},
            '매매지수': {'매매지수': {'color': 'blue', 'style': '-'}},
            '전세지수': {'전세지수': {'color': 'red', 'style': '-'}},
            '매매증감': {'매매증감': {'color': 'blue', 'style': 'bar'}},
            '전세증감': {'전세증감': {'color': 'red', 'style': 'bar'}}
        }

        # 시트별 데이터 매핑
        sheet_mappings = {
            '매수심리': {'file_type': '주간', 'sheet': '7.매수매도', 'columns': ['매도자많음', '매수자많음']},
            '전세현황': {'file_type': '주간', 'sheet': '9.전세수급', 'columns': ['수요>공급', '수요<공급']},
            '전세수급': {'file_type': '주간', 'sheet': '9.전세수급', 'columns': ['전세수급지수']},
            '매매전망': {'file_type': '월간', 'sheet': '25.KB부동산 매매가격 전망지수', 'columns': ['약간상승', '약간하락']},
            '전세전망': {'file_type': '월간', 'sheet': '26.KB부동산 전세가격 전망지수', 'columns': ['약간상승', '약간하락']},
            '매매지수': {'file_type': '주간', 'sheet': '3.매매지수', 'columns': ['지수']},
            '전세지수': {'file_type': '주간', 'sheet': '4.전세지수', 'columns': ['지수']},
            '매매증감': {'file_type': '주간', 'sheet': '1.매매증감', 'columns': ['증감']},
            '전세증감': {'file_type': '주간', 'sheet': '2.전세증감', 'columns': ['증감']}
        }

        plotted_data = False
        
        for chart_type in selected_charts:
            if chart_type not in sheet_mappings:
                continue
                
            mapping = sheet_mappings[chart_type]
            target_file = None
            
            # 해당 파일 타입 찾기
            for file_type, file_path in available_files:
                if file_type == mapping['file_type']:
                    target_file = file_path
                    break
            
            if not target_file:
                print(f"'{chart_type}'에 필요한 {mapping['file_type']} 파일을 찾을 수 없습니다.")
                continue

            try:
                # Excel 파일에서 특정 시트 읽기
                df = pd.read_excel(target_file, sheet_name=mapping['sheet'])
                
                # 데이터 구조 확인 및 처리
                if df.shape[0] < 3 or df.shape[1] < 2:
                    print(f"'{chart_type}' 데이터 형식이 올바르지 않습니다.")
                    continue
                
                # 1열: 날짜, 2행: 지역명, 3행부터: 실제 데이터
                date_column = df.iloc[:, 0]  # 1열 전체
                region_row = df.iloc[1, :]   # 2행 전체
                
                # 선택된 지역의 열 인덱스 찾기
                target_region = f"{sido} {sigungu}".strip()
                region_col_idx = None
                
                for idx, region_name in enumerate(region_row):
                    if pd.notna(region_name) and (sido in str(region_name) and sigungu in str(region_name)):
                        region_col_idx = idx
                        break
                
                if region_col_idx is None:
                    print(f"'{target_region}' 지역을 '{chart_type}' 데이터에서 찾을 수 없습니다.")
                    continue
                
                # 날짜 데이터 처리 (3행부터)
                dates = []
                values = {}
                
                for col_name in mapping['columns']:
                    values[col_name] = []
                
                for row_idx in range(2, len(df)):  # 3행부터 시작
                    date_val = date_column.iloc[row_idx]
                    if pd.isna(date_val):
                        continue
                        
                    try:
                        # 날짜 파싱
                        if isinstance(date_val, str):
                            if len(date_val) == 8:  # YYYYMMDD
                                parsed_date = datetime.strptime(date_val, "%Y%m%d")
                            elif len(date_val) == 6:  # YYYYMM
                                parsed_date = datetime.strptime(date_val + "01", "%Y%m%d")
                            else:
                                continue
                        else:
                            parsed_date = pd.to_datetime(date_val)
                        
                        # 기간 필터링
                        if not (start_date <= parsed_date <= end_date):
                            continue
                            
                        dates.append(parsed_date)
                        
                        # 데이터 값 추출
                        if chart_type in ['매매전망', '전세전망']:
                            # 전망 데이터는 약간상승 - 약간하락 계산
                            try:
                                rise_val = pd.to_numeric(df.iloc[row_idx, region_col_idx], errors='coerce')
                                fall_val = pd.to_numeric(df.iloc[row_idx, region_col_idx + 1], errors='coerce')  # 다음 열
                                if pd.notna(rise_val) and pd.notna(fall_val):
                                    diff_val = rise_val - fall_val
                                    values['전망차이'].append(diff_val)
                                else:
                                    values['전망차이'].append(0)
                            except:
                                values['전망차이'].append(0)
                        else:
                            # 일반 데이터 처리
                            for col_idx, col_name in enumerate(mapping['columns']):
                                try:
                                    val = pd.to_numeric(df.iloc[row_idx, region_col_idx + col_idx], errors='coerce')
                                    values[col_name].append(val if pd.notna(val) else 0)
                                except:
                                    values[col_name].append(0)
                    
                    except Exception as e:
                        print(f"날짜 파싱 오류 ({chart_type}): {e}")
                        continue
                
                # 차트 그리기
                if dates and any(values.values()):
                    style_info = chart_styles.get(chart_type, {})
                    
                    for col_name, data_values in values.items():
                        if not data_values:
                            continue
                            
                        col_style = style_info.get(col_name, {'color': 'gray', 'style': '-'})
                        
                        if col_style['style'] == 'bar':
                            # 막대 차트
                            self.timeseries_ax.bar(dates, data_values, 
                                                 label=f"{chart_type}_{col_name}",
                                                 color=col_style['color'],
                                                 alpha=0.7, width=20)
                        else:
                            # 꺾은선 차트
                            self.timeseries_ax.plot(dates, data_values,
                                                  label=f"{chart_type}_{col_name}",
                                                  color=col_style['color'],
                                                  linestyle=col_style['style'],
                                                  linewidth=2,
                                                  marker='o',
                                                  markersize=4)
                    
                    plotted_data = True
                        
            except Exception as e:
                print(f"'{chart_type}' 데이터 처리 중 오류: {e}")
                continue
        
        # 차트 설정
        if plotted_data:
            self.timeseries_ax.set_title(f"{sido} {sigungu} 다중 시계열 분석")
            self.timeseries_ax.set_xlabel("날짜")
            self.timeseries_ax.set_ylabel("값")
            self.timeseries_ax.grid(True, alpha=0.3)
            self.timeseries_ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
            
            # X축 날짜 포맷팅
            self.timeseries_ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
            self.timeseries_ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
            plt.setp(self.timeseries_ax.xaxis.get_majorticklabels(), rotation=45)
        else:
            self.timeseries_ax.text(0.5, 0.5, "선택한 조건에 해당하는 데이터가 없습니다.", 
                                   ha='center', va='center', transform=self.timeseries_ax.transAxes,
                                   fontsize=12)
        
        self.timeseries_fig.tight_layout()
        self.timeseries_canvas.draw()

    def _update_selected_apt_tree_ui(self):
        """선택 단지 목록 Treeview 업데이트 (시세차트 페이지)"""
        # 기존 목록 비우기
        for item in self.selected_apt_tree.get_children():
            self.selected_apt_tree.delete(item)

        # self.selected_apartments의 내용으로 Treeview 다시 채우기
        for item_id, apt_info in self.selected_apartments.items():
            self.selected_apt_tree.insert('', tk.END, values=(
                apt_info.get('지역', ''), apt_info.get('단지명', ''),
                apt_info.get('전용면적', ''), apt_info.get('연차', '')
            ), iid=item_id)

    def on_selected_apt_tree_select(self, event):
        """시세차트 선택 단지 Treeview 선택 이벤트 핸들러"""
        selected_item_id = self.selected_apt_tree.focus()
        if selected_item_id and selected_item_id in self.selected_apartments:
            apt_info = self.selected_apartments[selected_item_id]
            complex_serial_no = apt_info.get('단지기본일련번호')
            area_serial_no = apt_info.get('면적일련번호')
            self.current_selected_apt_info = apt_info
            if complex_serial_no and area_serial_no:
                self.loading_label.config(text=f"'{apt_info.get('단지명')} {apt_info.get('전용면적')}㎡' 시세 데이터를 조회 중입니다...")
                
                # --- 수정 시작 (네이버 단지 코드 및 면적 목록 로드 로직 추가) ---
                # 이 부분을 스레드로 실행하여 UI 멈춤을 방지
                thread_naver_danji = threading.Thread(target=self.fetch_naver_complex_code_and_update_ui, 
                                                      args=(complex_serial_no,))
                thread_naver_danji.daemon = True
                thread_naver_danji.start()
                # --- 수정 끝 ---

                # 기존 시세 차트 및 실거래 데이터 로드 로직
                thread_chart = threading.Thread(target=self.fetch_chart_data, args=(complex_serial_no, area_serial_no))
                thread_chart.daemon = True
                thread_chart.start()
                thread_real_tran = threading.Thread(target=self.fetch_real_transaction_data, args=(complex_serial_no, area_serial_no))
                thread_real_tran.daemon = True
                thread_real_tran.start()
            else:
                messagebox.showwarning("경고", "선택된 단지의 단지기본일련번호 또는 면적일련번호가 없습니다.")
                self.loading_label.config(text="데이터 조회 실패: 필수 정보 누락")
        else:
            self.loading_label.config(text="단지를 선택해주세요.")
            self.current_selected_apt_info = None

    def fetch_chart_data(self, complex_serial_no, area_serial_no):
        """KB 시세 차트 데이터 가져오기 (update_chart_display 호출)"""
        try:
            end_date = datetime.now()
            start_date_api = "19700101"
            end_date_api = end_date.strftime("%Y%m%d")

            base_url = "https://api.kbland.kr/land-price/price/PerMn/IntgrationChart"
            params = {
                "단지기본일련번호": complex_serial_no, "면적일련번호": area_serial_no,
                "거래구분": "0", "조회구분": "2",
                "조회시작일": start_date_api, "조회종료일": end_date_api
            }
            url = base_url + "?" + "&".join([f"{k}={quote(str(v))}" for k, v in params.items()])

            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
                'Accept': 'application/json, text/plain, */*',
                'Accept-Language': 'ko-KR,ko;q=0.9,en;q=0.8',
                'Accept-Encoding': 'gzip, deflate, br',
                'Connection': 'keep-alive',
                'Referer': 'https://kbland.kr/',
                'Origin': 'https://kbland.kr'
            }
            session = requests.Session()
            session.headers.update(headers)

            max_retries = 3
            for attempt in range(max_retries):
                try:
                    response = session.get(url, timeout=30)
                    response.raise_for_status()
                    break
                except requests.exceptions.ConnectionError as e:
                    if attempt == max_retries - 1:
                        raise e
                    time.sleep(2)

            if response.headers.get('content-type', '').startswith('application/json'):
                data = response.json()
            else:
                raise ValueError(f"JSON 응답이 아닙니다. Content-Type: {response.headers.get('content-type')}")

            self.root.after(0, self.update_chart_display, data)

        except Exception as e:
            error_msg = f"Chart API 오류: {str(e)}"
            self.root.after(0, self.show_error, error_msg)

    def fetch_real_transaction_data(self, complex_serial_no, area_serial_no):
        """KB 실거래 정보 데이터 가져오기 (parse_real_transaction_data 호출)"""
        try:
            base_url = "https://api.kbland.kr/land-price/price/LatestRealTranPrc"
            params = {
                "단지기본일련번호": complex_serial_no,
                "면적일련번호": area_serial_no,
                "거래구분": "0",  # 0: 전체
                "조회구분": "0",  # 0: 전체
                "첫페이지갯수": "100000",  # 충분히 큰 값으로 모든 데이터 가져오기 시도
                "현재페이지": "1",
                "페이지갯수": "0"
            }
            url = base_url + "?" + "&".join([f"{k}={quote(str(v))}" for k, v in params.items()])

            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
                'Accept': 'application/json, text/plain, */*',
                'Accept-Language': 'ko-KR,ko;q=0.9,en;q=0.8',
                'Accept-Encoding': 'gzip, deflate, br',
                'Connection': 'keep-alive',
                'Referer': 'https://kbland.kr/',
                'Origin': 'https://kbland.kr'
            }
            session = requests.Session()
            session.headers.update(headers)

            max_retries = 3
            for attempt in range(max_retries):
                try:
                    response = session.get(url, timeout=30)
                    response.raise_for_status()
                    break
                except requests.exceptions.ConnectionError as e:
                    if attempt == max_retries - 1:
                        raise e
                    time.sleep(2)

            if response.headers.get('content-type', '').startswith('application/json'):
                data = response.json()
            else:
                raise ValueError("실거래 API JSON 응답이 아닙니다.")
            
            sale_list, lease_list = self.parse_real_transaction_data(data)
            self.root.after(0, self.display_real_transaction_data, sale_list, lease_list)

        except Exception as e:
            self.root.after(0, self.show_error, f"실거래 API 오류: {e}")

    def parse_real_transaction_data(self, raw_data):
        """KB 실거래 정보 API 응답 데이터 파싱 및 분류"""
        sale_list = []
        lease_list = []
        items = raw_data.get('dataBody', {}).get('data', []) 
        
        for item in items:
            물건거래명 = item.get('물건거래명', '')
            계약년월 = item.get('계약년월', '')
            계약일 = item.get('계약일', '')
            해당층수 = item.get('해당층수', '-')
            
            날짜 = f"{계약년월[:4]}-{계약년월[4:]}-{계약일}" if 계약년월 and 계약일 else '-'
            
            거래금액_표시 = '-'
            if 물건거래명 == '매매':
                금액_str = str(item.get('매매실거래금액', '0')).replace(',', '')
                거래금액_표시 = f"{float(금액_str):.0f} 만원" if 금액_str and float(금액_str) > 0 else '-'
                sale_list.append((날짜, 거래금액_표시, 해당층수))
            elif 물건거래명 == '전세':
                금액_str = str(item.get('전세실거래금액', '0')).replace(',', '')
                거래금액_표시 = f"{float(금액_str):.0f} 만원" if 금액_str and float(금액_str) > 0 else '-'
                lease_list.append((날짜, 거래금액_표시, 해당층수))
            elif 물건거래명 == '월세':
                보증금액_str = str(item.get('보증금액', '0')).replace(',', '')
                월세금액_str = str(item.get('월세금액', '0')).replace(',', '')
                보증금 = f"{float(보증금액_str):.0f}" if 보증금액_str and float(보증금액_str) > 0 else '0'
                월세 = f"{float(월세금액_str):.0f}" if 월세금액_str and float(월세금액_str) > 0 else '0'
                거래금액_표시 = f"{보증금}/{월세} 만원"
                lease_list.append((날짜, 거래금액_표시, 해당층수))
            
        sale_list.sort(key=lambda x: x[0], reverse=True)
        lease_list.sort(key=lambda x: x[0], reverse=True)

        return sale_list, lease_list

    def display_real_transaction_data(self, sale_list, lease_list):
        """실거래 정보를 Treeview에 표시"""
        # 기존 데이터 삭제
        for item in self.sale_real_tran_tree.get_children():
            self.sale_real_tran_tree.delete(item)
        for item in self.lease_real_tran_tree.get_children():
            self.lease_real_tran_tree.delete(item)
        
        # 새 데이터 삽입
        for row_data in sale_list:
            self.sale_real_tran_tree.insert('', tk.END, values=row_data)
        for row_data in lease_list:
            self.lease_real_tran_tree.insert('', tk.END, values=row_data)

    def update_chart_display(self, chart_data_raw):
        """시세 차트 화면 업데이트 (update_chart_view 호출)"""
        parsed_chart_data = self.parse_chart_api_data(chart_data_raw)
        self.current_chart_data = parsed_chart_data

        if parsed_chart_data and (parsed_chart_data['sise_data'] or \
           parsed_chart_data['sale_real_data'] or parsed_chart_data['lease_real_data']):

            all_dates_in_chart_data = []
            if parsed_chart_data['sise_data']:
                all_dates_in_chart_data.extend([pd.to_datetime(d['기준년월'], format='%Y%m') for d in parsed_chart_data['sise_data']])
            if parsed_chart_data['sale_real_data']:
                all_dates_in_chart_data.extend([pd.to_datetime(d['기준년월일'], format='%Y%m%d') for d in parsed_chart_data['sale_real_data'] if d['거래금액'] is not None])
            if parsed_chart_data['lease_real_data']:
                all_dates_in_chart_data.extend([pd.to_datetime(d['기준년월일'], format='%Y%m%d') for d in parsed_chart_data['lease_real_data'] if d['거래금액'] is not None])

            if all_dates_in_chart_data:
                min_date = min(all_dates_in_chart_data)
                max_date = max(all_dates_in_chart_data)
                self.chart_start_date_var.set(min_date.strftime("%Y-%m-%d"))
                self.chart_end_date_var.set(max_date.strftime("%Y-%m-%d"))
            else:
                self.chart_start_date_var.set("")
                self.chart_end_date_var.set("")

            self.update_chart_view()
            self.loading_label.config(text="시세 차트가 업데이트되었습니다.")
        else:
            self.fig.clf()
            gs = self.fig.add_gridspec(nrows=2, ncols=1, height_ratios=[3, 1])
            self.ax = self.fig.add_subplot(gs[0, 0])
            self.ax2 = self.fig.add_subplot(gs[1, 0], sharex=self.ax)
            self.ax.text(0.5, 0.5, "데이터가 없습니다.", ha='center', va='center', transform=self.ax.transAxes)
            self.canvas.draw()
            self.update_chart_summary_info(None)
            self.loading_label.config(text="선택된 단지의 시세 데이터를 찾을 수 없습니다.")
            self.chart_start_date_var.set("")
            self.chart_end_date_var.set("")

    def update_chart_view(self):
        """차트 옵션 및 기간에 따라 차트 뷰 업데이트 (draw_price_chart, update_chart_summary_info 호출)"""
        if not self.current_chart_data:
            self.loading_label.config(text="차트 데이터가 없습니다.")
            return

        filtered_sise_data = []
        filtered_sale_real_data = []
        filtered_lease_real_data = []

        try:
            chart_start_date = datetime.strptime(self.chart_start_date_var.get(), "%Y-%m-%d").date()
            chart_end_date = datetime.strptime(self.chart_end_date_var.get(), "%Y-%m-%d").date()
            if chart_start_date > chart_end_date:
                messagebox.showwarning("경고", "차트 시작일은 종료일보다 빨라야 합니다.")
                return
        except ValueError:
            messagebox.showwarning("경고", "차트 기간 형식이 올바르지 않습니다 (YYYY-MM-DD).")
            return

        for item in self.current_chart_data['sise_data']:
            item_date = datetime.strptime(item['기준년월'], "%Y%m").date()
            if chart_start_date <= item_date <= chart_end_date:
                filtered_sise_data.append(item)

        for item in self.current_chart_data['sale_real_data']:
            item_date = datetime.strptime(item['기준년월일'], "%Y%m%d").date()
            if chart_start_date <= item_date <= chart_end_date:
                filtered_sale_real_data.append(item)

        for item in self.current_chart_data['lease_real_data']:
            item_date = datetime.strptime(item['기준년월일'], "%Y%m%d").date()
            if chart_start_date <= item_date <= chart_end_date:
                filtered_lease_real_data.append(item)

        self.draw_price_chart(
            {'sise_data': filtered_sise_data, 'sale_real_data': filtered_sale_real_data, 'lease_real_data': filtered_lease_real_data},
            show_sise_sale=self.show_sise_sale_var.get(),
            show_sise_lease=self.show_sise_lease_var.get(),
            show_sise_ratio=self.show_sise_ratio_var.get(),
            show_real_sale=self.show_real_sale_var.get(),
            show_real_lease=self.show_real_lease_var.get(),
            apt_info=self.current_selected_apt_info  # 단지 정보 전달
        )
        self.update_chart_summary_info(
            {'sise_data': filtered_sise_data, 'sale_real_data': filtered_sale_real_data, 'lease_real_data': filtered_lease_real_data}
        )

    def parse_chart_api_data(self, chart_data_raw):
        """시세 차트 API 응답 데이터 파싱"""
        sise_data = []
        sale_real_data = []
        lease_real_data = []
        try:
            data_body_data = chart_data_raw.get('dataBody', {}).get('data', {})

            sise_groups = data_body_data.get('시세', [])
            for group in sise_groups:
                if 'items' in group and isinstance(group['items'], list):
                    for item in group['items']:
                        sise_data.append({
                            '기준년월': item.get('기준년월', ''),
                            '매매일반거래가': item.get('매매일반거래가', None),
                            '전세일반거래가': item.get('전세일반거래가', None),
                            '전세가율': item.get('전세가율', None)
                        })
                        if '매매실거래금액리스트' in item and isinstance(item['매매실거래금액리스트'], list):
                            기준년월 = item.get('기준년월', '')
                            for 금액_str in item['매매실거래금액리스트']:
                                거래금액 = float(str(금액_str).replace(',', '')) if str(금액_str).replace(',', '') else None
                                if 거래금액 is not None:
                                    sale_real_data.append({'기준년월일': 기준년월 + '01', '거래금액': 거래금액})
                        if '전세실거래금액리스트' in item and isinstance(item['전세실거래금액리스트'], list):
                            기준년월 = item.get('기준년월', '')
                            for 금액_str in item['전세실거래금액리스트']:
                                거래금액 = float(str(금액_str).replace(',', '')) if str(금액_str).replace(',', '') else None
                                if 거래금액 is not None:
                                    lease_real_data.append({'기준년월일': 기준년월 + '01', '거래금액': 거래금액})
            sise_data.sort(key=lambda x: x['기준년월'])

            sale_real_df = pd.DataFrame(sale_real_data)
            if not sale_real_df.empty:
                sale_real_data = sale_real_df.drop_duplicates(subset=['기준년월일', '거래금액']).to_dict(orient='records')
            lease_real_df = pd.DataFrame(lease_real_data)
            if not lease_real_df.empty:
                lease_real_data = lease_real_df.drop_duplicates(subset=['기준년월일', '거래금액']).to_dict(orient='records')

            sale_real_data.sort(key=lambda x: x['기준년월일'])
            lease_real_data.sort(key=lambda x: x['기준년월일'])

            return {
                'sise_data': sise_data,
                'sale_real_data': sale_real_data,
                'lease_real_data': lease_real_data
            }
        except Exception as e:
            self.root.after(0, self.show_error, f"차트 데이터 파싱 오류: {e}")
            return None

    def draw_price_chart(self, chart_data, show_sise_sale=True, show_sise_lease=True, show_sise_ratio=True, show_real_sale=True, show_real_lease=True, apt_info=None):
        """시세/실거래 차트 그리기"""
        self.fig.clf()
        gs = self.fig.add_gridspec(nrows=2, ncols=1, height_ratios=[3, 1])
        self.ax = self.fig.add_subplot(gs[0, 0])
        self.ax2 = self.fig.add_subplot(gs[1, 0], sharex=self.ax)

        sise_data = chart_data.get('sise_data', [])
        sale_real_data = chart_data.get('sale_real_data', [])
        lease_real_data = chart_data.get('lease_real_data', [])

        if not (sise_data or sale_real_data or lease_real_data):
            self.ax.text(0.5, 0.5, "데이터가 없습니다.", ha='center', va='center', transform=self.ax.transAxes)
            self.canvas.draw()
            return

        prices_ax = self.ax
        ratio_ax = prices_ax.twinx()

        # 시세 데이터 플로팅
        if sise_data:
            dates = [pd.to_datetime(d['기준년월'], format='%Y%m') for d in sise_data]
            prices_ax.set_ylabel('가격 (만원)', color='blue')
            prices_ax.tick_params(axis='y', labelcolor='blue')
            if show_sise_sale:
                prices_ax.plot(dates, [d['매매일반거래가'] for d in sise_data], label='매매일반거래가', color='blue', linestyle='solid', linewidth=2, marker='None')
            if show_sise_lease:
                prices_ax.plot(dates, [d['전세일반거래가'] for d in sise_data], label='전세일반거래가', color='red', linestyle='solid', linewidth=2, marker='None')

            ratio_ax.set_ylabel('전세가율 (%)', color='green')
            ratio_ax.tick_params(axis='y', labelcolor='green')
            if show_sise_ratio:
                ratio_ax.plot(dates, [d['전세가율'] for d in sise_data], label='전세가율', color='green', linestyle=':', linewidth=2, marker='None')
        else:
            prices_ax.set_ylabel('가격 (만원)', color='blue')
            prices_ax.tick_params(axis='y', labelcolor='blue')
            ratio_ax.set_ylabel('전세가율 (%)', color='green')
            ratio_ax.tick_params(axis='y', labelcolor='green')

        # Y축 격자선 설정
        prices_ax.grid(True, which='major', axis='y', linestyle='--', linewidth=0.5, color='gray')
        prices_ax.grid(True, which='minor', axis='y', linestyle=':', linewidth=0.25, color='lightgray')

        # 실거래가 데이터 플로팅 (점도표)
        if show_real_sale and sale_real_data:
            sale_real_dates = [pd.to_datetime(d['기준년월일'], format='%Y%m%d') for d in sale_real_data if d['거래금액'] is not None]
            sale_real_prices = [d['거래금액'] for d in sale_real_data if d['거래금액'] is not None]
            prices_ax.scatter(sale_real_dates, sale_real_prices, label='매매실거래', color='royalblue', marker='o', s=15, alpha=0.6)

        if show_real_lease and lease_real_data:
            lease_real_dates = [pd.to_datetime(d['기준년월일'], format='%Y%m%d') for d in lease_real_data if d['거래금액'] is not None]
            lease_real_prices = [d['거래금액'] for d in lease_real_data if d['거래금액'] is not None]
            prices_ax.scatter(lease_real_dates, lease_real_prices, label='전세실거래', color='firebrick', marker='s', s=15, alpha=0.6)

        # 하단 거래 건수 차트 그리기
        self.ax2.clear()
        bar_width = 15

        sale_counts = {}
        lease_counts = {}

        if show_real_sale:
            for d in sale_real_data:
                month = d['기준년월일'][:6]
                sale_counts[month] = sale_counts.get(month, 0) + 1

        if show_real_lease:
            for d in lease_real_data:
                month = d['기준년월일'][:6]
                lease_counts[month] = lease_counts.get(month, 0) + 1

        months = sorted(set(list(sale_counts.keys()) + list(lease_counts.keys())))
        x = [pd.to_datetime(m, format='%Y%m') for m in months]
        sale_y = [sale_counts.get(m, 0) for m in months]
        lease_y = [lease_counts.get(m, 0) for m in months]

        if show_real_sale and x:
            self.ax2.bar([d - pd.Timedelta(days=bar_width/2) for d in x], sale_y, width=bar_width, color='royalblue', label='매매 건수')
        if show_real_lease and x:
            self.ax2.bar([d + pd.Timedelta(days=bar_width/2) for d in x], lease_y, width=bar_width, color='firebrick', label='전세 건수')

        self.ax2.set_ylabel('거래 건수')
        self.ax2.grid(True, which='major', axis='y')
        self.ax2.legend(loc='upper left')
        self.ax2.yaxis.set_major_locator(ticker.MaxNLocator(integer=True))

        # X축 날짜 포맷 설정
        all_dates = []
        if sise_data and (show_sise_sale or show_sise_lease or show_sise_ratio):
            all_dates.extend([pd.to_datetime(d['기준년월'], format='%Y%m') for d in sise_data])
        if sale_real_data and show_real_sale:
            all_dates.extend([pd.to_datetime(d['기준년월일'], format='%Y%m%d') for d in sale_real_data if d['거래금액'] is not None])
        if lease_real_data and show_real_lease:
            all_dates.extend([pd.to_datetime(d['기준년월일'], format='%Y%m%d') for d in lease_real_data if d['거래금액'] is not None])

        if all_dates:
            min_date = min(all_dates) - pd.Timedelta(days=30)
            max_date = max(all_dates) + pd.Timedelta(days=30)
            self.ax.set_xlim(min_date, max_date)

            self.ax.xaxis.set_major_locator(mdates.MonthLocator(interval=6))
            self.ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
            self.ax.grid(True, which='major', axis='x', linestyle='--', linewidth=0.5, color='gray')

            self.ax.xaxis.set_minor_locator(mdates.MonthLocator(interval=1))
            self.ax.grid(True, which='minor', axis='x', linestyle=':', linewidth=0.25, color='lightgray')

            self.ax2.grid(True, which='major', axis='x', linestyle='--', linewidth=0.5, color='gray')
            self.ax2.grid(True, which='minor', axis='x', linestyle=':', linewidth=0.25, color='lightgray')

            plt.setp(self.ax.get_xticklabels(), visible=False)
            self.fig.autofmt_xdate(rotation=45)
        
        # 시세차트 범례
        lines_prices, labels_prices = prices_ax.get_legend_handles_labels()
        lines_ratio, labels_ratio = ratio_ax.get_legend_handles_labels()

        all_lines_for_ax = lines_prices + lines_ratio
        all_labels_for_ax = labels_prices + labels_ratio

        order_for_ax = ['매매일반거래가', '전세일반거래가', '전세가율', '매매실거래', '전세실거래']
        handles_labels_for_ax = []
        for h, l in zip(all_lines_for_ax, all_labels_for_ax):
            if l in order_for_ax:
                handles_labels_for_ax.append((h, l))

        if handles_labels_for_ax:
            handles_labels_for_ax.sort(key=lambda hl: order_for_ax.index(hl[1]))
            handles_ax, labels_ax = zip(*handles_labels_for_ax)
            self.ax.legend(handles_ax, labels_ax, loc='upper left')

        # 차트 제목 설정
        if apt_info:
            title_text = f"{apt_info.get('단지명', '단지명 없음')} - {apt_info.get('전용면적', '면적 없음')}㎡"
        else:
            title_text = "아파트 시세 및 실거래가 변화"
        self.ax.set_title(title_text)
        
        self.fig.tight_layout()
        self.canvas.draw()

    def update_chart_summary_info(self, chart_data):
        """차트 하단 요약 정보 업데이트"""
        # 레이블 초기화
        for label_key in self.chart_info_labels:
            self.chart_info_labels[label_key].config(text=f"{label_key}: -")

        if not chart_data:
            return

        sise_data = chart_data.get('sise_data', [])
        sale_real_data = chart_data.get('sale_real_data', [])
        lease_real_data = chart_data.get('lease_real_data', [])

        # 매매 시세 최고가/최근가
        max_sale_price_sise = None
        max_sale_price_sise_date = None
        latest_sale_price_sise = None
        if sise_data:
            latest_sale_price_sise = sise_data[-1].get('매매일반거래가')
            for item in sise_data:
                price = item.get('매매일반거래가')
                if price is not None:
                    if max_sale_price_sise is None or price > max_sale_price_sise:
                        max_sale_price_sise = price
                        max_sale_price_sise_date = datetime.strptime(item['기준년월'], "%Y%m") 
        self.chart_info_labels['매매최고가'].config(text=f"매매최고가: {max_sale_price_sise if max_sale_price_sise is not None else '-'} 만원 ({max_sale_price_sise_date.strftime('%Y-%m') if max_sale_price_sise_date else '-'})")
        self.chart_info_labels['매매최근가'].config(text=f"매매최근가: {latest_sale_price_sise if latest_sale_price_sise is not None else '-'} 만원")

        # 전세 시세 최고가/최근가
        max_lease_price_sise = None
        max_lease_price_sise_date = None 
        latest_lease_price_sise = None
        if sise_data:
            latest_lease_price_sise = sise_data[-1].get('전세일반거래가')
            for item in sise_data:
                price = item.get('전세일반거래가')
                if price is not None:
                    if max_lease_price_sise is None or price > max_lease_price_sise:
                        max_lease_price_sise = price
                        max_lease_price_sise_date = datetime.strptime(item['기준년월'], "%Y%m") 
        self.chart_info_labels['전세최고가'].config(text=f"전세최고가: {max_lease_price_sise if max_lease_price_sise is not None else '-'} 만원 ({max_lease_price_sise_date.strftime('%Y-%m') if max_lease_price_sise_date else '-'})") 
        self.chart_info_labels['전세최근가'].config(text=f"전세최근가: {latest_lease_price_sise if latest_lease_price_sise is not None else '-'} 만원")

        # 실거래 매매 최고가/최근가
        max_sale_price_real = None
        max_sale_price_real_date = None
        latest_sale_price_real = None
        if sale_real_data:
            latest_sale_price_real = sale_real_data[-1].get('거래금액')
            for item in sale_real_data:
                price = item.get('거래금액')
                if price is not None:
                    if max_sale_price_real is None or price > max_sale_price_real:
                        max_sale_price_real = price
                        max_sale_price_real_date = datetime.strptime(item['기준년월일'], "%Y%m%d")
        self.chart_info_labels['실거래매매최고가'].config(text=f"실거래매매최고가: {max_sale_price_real if max_sale_price_real is not None else '-'} 만원 ({max_sale_price_real_date.strftime('%Y-%m-%d') if max_sale_price_real_date else '-'})")
        self.chart_info_labels['실거래매매최근가'].config(text=f"실거래매매최근가: {latest_sale_price_real if latest_sale_price_real is not None else '-'} 만원")

        # 실거래 전세 최고가/최근가
        max_lease_price_real = None
        max_lease_price_real_date = None
        latest_lease_price_real = None
        if lease_real_data:
            latest_lease_price_real = lease_real_data[-1].get('거래금액')
            for item in lease_real_data:
                price = item.get('거래금액')
                if price is not None:
                    if max_lease_price_real is None or price > max_lease_price_real:
                        max_lease_price_real = price
                        max_lease_price_real_date = datetime.strptime(item['기준년월일'], "%Y%m%d")
        self.chart_info_labels['실거래전세최고가'].config(text=f"실거래전세최고가: {max_lease_price_real if max_lease_price_real is not None else '-'} 만원 ({max_lease_price_real_date.strftime('%Y-%m-%d') if max_lease_price_real_date else '-'})")
        self.chart_info_labels['실거래전세최근가'].config(text=f"실거래전세최근가: {latest_lease_price_real if latest_lease_price_real is not None else '-'} 만원")

        # 매매하락가, 매매하락율, 매매회복율
        decline_price = None
        decline_rate_str = "N/A"
        recovery_rate_str = "N/A"

        if max_sale_price_sise is not None and max_sale_price_sise_date is not None and latest_sale_price_sise is not None:
            sise_after_peak_values = []
            for item in sise_data:
                item_date = datetime.strptime(item['기준년월'], "%Y%m")
                if item_date >= max_sale_price_sise_date and item.get('매매일반거래가') is not None:
                    sise_after_peak_values.append(item.get('매매일반거래가'))
            
            if sise_after_peak_values:
                decline_price = min(sise_after_peak_values)
                
                if max_sale_price_sise > 0:
                    decline_rate = ((max_sale_price_sise - decline_price) / max_sale_price_sise) * 100
                    decline_rate_str = f"{decline_rate:.2f}%"
                else:
                    decline_rate_str = "N/A"

                if decline_price is not None and max_sale_price_sise is not None and (max_sale_price_sise - decline_price) > 0:
                    if latest_sale_price_sise > decline_price and latest_sale_price_sise < max_sale_price_sise:
                        recovery_rate = ((latest_sale_price_sise - decline_price) / (max_sale_price_sise - decline_price)) * 100
                        recovery_rate_str = f"{recovery_rate:.2f}%"
                    elif latest_sale_price_sise >= max_sale_price_sise:
                        recovery_rate_str = "100.00% (최고가 회복)"
                    else:
                        recovery_rate_str = "0.00%"
                else:
                    recovery_rate_str = "N/A"
            else:
                decline_price = max_sale_price_sise
                decline_rate_str = "0.00%"
                recovery_rate_str = "N/A"#excel
                
        self.chart_info_labels['매매하락가'].config(text=f"매매하락가: {decline_price if decline_price is not None else '-'} 만원")
        self.chart_info_labels['매매하락율'].config(text=f"매매하락율: {decline_rate_str}")
        self.chart_info_labels['매매회복율'].config(text=f"매매회복율: {recovery_rate_str}")

    def clear_charts(self):
        """차트 영역 초기화"""
        self.fig.clf()
        gs = self.fig.add_gridspec(nrows=2, ncols=1, height_ratios=[3, 1])
        self.ax = self.fig.add_subplot(gs[0, 0])
        self.ax2 = self.fig.add_subplot(gs[1, 0], sharex=self.ax)
        self.ax.text(0.5, 0.5, "단지를 선택하여 차트를 로드하세요.", ha='center', va='center', transform=self.ax.transAxes)
        self.canvas.draw()
        self.update_chart_summary_info(None) # 차트 요약 정보도 초기화

    def clear_info(self):
        """정보 패널 초기화"""
        for item in self.sale_real_tran_tree.get_children():
            self.sale_real_tran_tree.delete(item)
        for item in self.lease_real_tran_tree.get_children():
            self.lease_real_tran_tree.delete(item)
        self.update_chart_summary_info(None) # 차트 요약 정보 초기화

# --- 삭제 시작 (load_naver_sido_list 메서드 전체) ---
    # def load_naver_sido_list(self):
    #     """네이버 부동산 시도 목록 로드"""
    #     url = "https://m.land.naver.com/map/getRegionList?cortarNo=0000000000&mycortarNo="
    #     data = self.fetch_naver_api_data(url) 
    #     if data and 'result' in data and 'list' in data['result']:
    #         sido_list_raw = data['result']['list']
    #         self.naver_sido_map = {item['CortarNm']: item['CortarNo'] for item in sido_list_raw}
    #         sido_names = list(self.naver_sido_map.keys())
    #         self.naver_sido_combobox['values'] = sido_names
            
    #         # set() 호출 전에 이벤트 바인딩 해제하여 연쇄 호출 방지
    #         self.naver_sido_combobox.unbind("<<ComboboxSelected>>")
    #         if sido_names:
    #             self.naver_sido_combobox.set(sido_names[0]) 
    #             # 시도 선택 후 시군구 목록을 자동으로 로드 (이벤트 핸들러를 직접 호출)
    #             self.on_naver_sido_selected(None) 
    #         else:
    #             self.naver_sido_combobox.set("시도 선택")
    #         self.naver_sido_combobox.bind("<<ComboboxSelected>>", self.on_naver_sido_selected) # 다시 바인딩
    #     else:
    #         print("네이버 시도 목록 로드 실패")
    #         self.naver_sido_combobox['values'] = ["목록 로드 실패"]
    #         self.naver_sido_combobox.set("목록 로드 실패")
    #     self.root.update_idletasks() # UI 갱신 강제
# --- 삭제 끝 ---

# --- 삭제 시작 (on_naver_sido_selected 메서드 전체) ---
    # def on_naver_sido_selected(self, event):
    #     """네이버 시도 선택 시 시군구 목록 로드"""
    #     selected_sido_name = self.naver_sido_combobox.get()
    #     selected_sido_code = self.naver_sido_map.get(selected_sido_name)
        
    #     print(f"on_naver_sido_selected - 선택된 시도명: {selected_sido_name}, 코드: {selected_sido_code}")
        
    #     if selected_sido_code:
    #         self.load_naver_sigungu_list(selected_sido_code)
    #         # 시군구는 로드되었으니, 그 하위(법정동, 단지, 면적)만 초기화
    #         self.clear_naver_lower_combos(['bjdong', 'danji', 'myeoneok'])
# --- 삭제 끝 ---

# --- 삭제 시작 (load_naver_sigungu_list 메서드 전체) ---
    # def load_naver_sigungu_list(self, sido_code):
    #     """네이버 시군구 목록 로드"""
    #     url = f"https://m.land.naver.com/map/getRegionList?cortarNo={sido_code}&mycortarNo={sido_code}"
        
    #     print(f"load_naver_sigungu_list - 요청 URL: {url}")
        
    #     data = self.fetch_naver_api_data(url) 
        
    #     print(f"load_naver_sigungu_list - fetch_naver_api_data 응답 데이터: {data}")

    #     if data and 'result' in data and 'list' in data['result']:
    #         sigungu_list_raw = data['result']['list']
            
    #         print(f"load_naver_sigungu_list - 필터링 전 시군구 raw 리스트: {sigungu_list_raw}")
            
    #         self.naver_sigungu_map = {item['CortarNm']: item['CortarNo'] for item in sigungu_list_raw}
    #         sigungu_names = list(self.naver_sigungu_map.keys()) # 시군구 이름 리스트
    #         self.naver_sigungu_combobox['values'] = sigungu_names
            
    #         print(f"load_naver_sigungu_list - 최종 시군구 목록: {sigungu_names}")
            
    #         # set() 호출 전에 이벤트 바인딩 해제
    #         self.naver_sigungu_combobox.unbind("<<ComboboxSelected>>")
    #         if sigungu_names: # 목록이 비어있지 않다면 첫 번째 항목을 선택
    #             self.naver_sigungu_combobox.set(sigungu_names[0])
    #         else: # 목록이 비어있다면 "시군구 선택"으로 설정
    #             self.naver_sigungu_combobox.set("시군구 선택")
    #         self.naver_sigungu_combobox.bind("<<ComboboxSelected>>", self.on_naver_sigungu_selected) # 다시 바인딩
    #         self.root.update_idletasks() # UI 갱신 강제
    #     else:
    #         print(f"네이버 시군구 목록 로드 실패 (시도 코드: {sido_code}) - 응답 데이터 문제 또는 형식 불일치")
    #         self.naver_sigungu_combobox['values'] = ["목록 로드 실패"]
    #         self.naver_sigungu_combobox.set("목록 로드 실패") # 실패 시에도 명시적으로 설정
    #         self.root.update_idletasks() # UI 갱신 강제
# --- 삭제 끝 ---

# --- 삭제 시작 (on_naver_sigungu_selected 메서드 전체) ---
    # def on_naver_sigungu_selected(self, event):
    #     """네이버 시군구 선택 시 법정동 목록 로드"""
    #     selected_sigungu_name = self.naver_sigungu_combobox.get()
    #     selected_sigungu_code = self.naver_sigungu_map.get(selected_sigungu_name)
        
    #     print(f"on_naver_sigungu_selected - 선택된 시군구명: {selected_sigungu_name}, 코드: {selected_sigungu_code}")
        
    #     if selected_sigungu_code:
    #         self.load_naver_bjdong_list(selected_sigungu_code)
    #         # 법정동은 로드되었으니, 그 하위(단지, 면적)만 초기화
    #         self.clear_naver_lower_combos(['danji', 'myeoneok'])
# --- 삭제 끝 ---

# --- 삭제 시작 (load_naver_bjdong_list 메서드 전체) ---
    # def load_naver_bjdong_list(self, sigungu_code):
    #     """네이버 부동산 법정동 목록 로드"""
    #     url = f"https://m.land.naver.com/map/getRegionList?cortarNo={sigungu_code}&mycortarNo={sigungu_code}"
        
    #     print(f"load_naver_bjdong_list - 요청 URL: {url}")
        
    #     data = self.fetch_naver_api_data(url) 

    #     print(f"load_naver_bjdong_list - fetch_naver_api_data 응답 데이터: {data}")

    #     if data and 'result' in data and 'list' in data['result']:
    #         bjdong_list_raw = data['result']['list']
            
    #         print(f"load_naver_bjdong_list - 필터링 전 법정동 raw 리스트: {bjdong_list_raw}")
            
    #         self.naver_bjdong_map = {item['CortarNm']: item['CortarNo'] for item in bjdong_list_raw}
    #         bjdong_names = list(self.naver_bjdong_map.keys()) # 법정동 이름 리스트
    #         self.naver_bjdong_combobox['values'] = bjdong_names
            
    #         print(f"load_naver_bjdong_list - 최종 법정동 목록: {bjdong_names}")
            
    #         # set() 호출 전에 이벤트 바인딩 해제
    #         self.naver_bjdong_combobox.unbind("<<ComboboxSelected>>")
    #         if bjdong_names:
    #             self.naver_bjdong_combobox.set(bjdong_names[0])
    #         else:
    #             self.naver_bjdong_combobox.set("법정동 선택")
    #         self.naver_bjdong_combobox.bind("<<ComboboxSelected>>", self.on_naver_bjdong_selected) # 다시 바인딩
    #         self.root.update_idletasks() # UI 갱신 강제
    #     else:
    #         print(f"네이버 법정동 목록 로드 실패 (시군구 코드: {sigungu_code})")
    #         self.naver_bjdong_combobox['values'] = ["목록 로드 실패"]
    #         self.naver_bjdong_combobox.set("목록 로드 실패")
    #         self.root.update_idletasks() # UI 갱신 강제
# --- 삭제 끝 ---
    
# --- 삭제 시작 (on_naver_bjdong_selected 메서드 전체) ---
    # def on_naver_bjdong_selected(self, event):
    #     """네이버 법정동 선택 시 단지 목록 로드"""
    #     selected_bjdong_name = self.naver_bjdong_combobox.get()
    #     selected_bjdong_code = self.naver_bjdong_map.get(selected_bjdong_name)
        
    #     print(f"on_naver_bjdong_selected - 선택된 법정동명: {selected_bjdong_name}, 코드: {selected_bjdong_code}")
        
    #     if selected_bjdong_code:
    #         self.load_naver_danji_list(selected_bjdong_code)
    #         # 단지는 로드되었으니, 그 하위(면적)만 초기화
    #         self.clear_naver_lower_combos(['myeoneok'])
# --- 삭제 끝 ---

# --- 삭제 시작 (load_naver_danji_list 메서드 전체) ---
    # def load_naver_danji_list(self, bjdong_code):
    #     """네이버 부동산 단지 목록 로드"""
    #     url = f"https://m.land.naver.com/complex/ajax/complexListByCortarNo={bjdong_code}"
    #     print(f"load_naver_danji_list - 요청 URL: {url}")
    #     data = self.fetch_naver_api_data(url)
    #     print(f"load_naver_danji_list - fetch_naver_api_data 응답 데이터: {data}")
        
    #     if data and 'result' in data and isinstance(data['result'], list):
    #         danji_list_raw = data['result']
    #         self.naver_danji_map = {}
    #         danji_display_list = []

    #         for item in danji_list_raw:
    #             hscp_no = item.get('hscpNo')
    #             hscp_nm = item.get('hscpNm')
    #             hscp_type_nm = item.get('hscpTypeNm')

    #             if hscp_no and hscp_nm and hscp_type_nm:
    #                 display_text = f"{hscp_nm}({hscp_type_nm})"
    #                 self.naver_danji_map[display_text] = hscp_no
    #                 danji_display_list.append(display_text)
            
    #         print(f"load_naver_danji_list - 생성된 단지 맵: {self.naver_danji_map}")
    #         print(f"load_naver_danji_list - 콤보박스에 채워질 단지 목록: {danji_display_list}")

    #         self.naver_danji_combobox['values'] = danji_display_list
            
    #         # set() 호출 전에 이벤트 바인딩 해제
    #         self.naver_danji_combobox.unbind("<<ComboboxSelected>>")
    #         if danji_display_list:
    #             self.naver_danji_combobox.set(danji_display_list[0])
    #         else:
    #             self.naver_danji_combobox.set("단지 선택")
    #         self.naver_danji_combobox.bind("<<ComboboxSelected>>", self.on_naver_danji_selected) # 다시 바인딩
    #         self.root.update_idletasks() # UI 갱신 강제
    #     else:
    #         print(f"네이버 단지 목록 로드 실패 (법정동 코드: {bjdong_code})")
    #         self.naver_danji_combobox['values'] = ["목록 로드 실패"]
    #         self.naver_danji_combobox.set("목록 로드 실패")
    #         self.root.update_idletasks() # UI 갱신 강제
# --- 삭제 끝 ---

# --- 삭제 시작 (on_naver_danji_selected 메서드 전체) ---
    # def on_naver_danji_selected(self, event=None, naver_danji_code_param=None):
    #     """
    #     네이버 단지 선택 시 면적 목록 로드.
    #     event: 콤보박스 선택 이벤트 (UI에서 선택 시)
    #     naver_danji_code_param: 직접 코드 값을 넘겨 호출할 때 (프로그램에서 자동 선택 시)
    #     """
    #     if naver_danji_code_param: # 매개변수로 직접 코드 값이 넘어왔을 경우
    #         selected_danji_code = naver_danji_code_param
    #         # 맵에서 단지명 찾기 (디버깅/로그용, 필수 아님)
    #         selected_danji_name = next((name for name, code in self.naver_danji_map.items() if code == naver_danji_code_param), f"Code_{naver_danji_code_param}")
    #         print(f"on_naver_danji_selected (프로그램 호출) - 단지명: {selected_danji_name}, 코드: {selected_danji_code}")
    #     else: # 콤보박스 선택 이벤트에 의해 호출된 경우
    #         selected_danji_name = self.naver_danji_combobox.get()
    #         selected_danji_code = self.naver_danji_map.get(selected_danji_name)
    #         print(f"on_naver_danji_selected (UI 호출) - 선택된 단지명: {selected_danji_name}, 코드: {selected_danji_code}")
        
    #     if selected_danji_code:
    #         self.load_naver_myeoneok_list(selected_danji_code)
    #         # 면적은 가장 하위 레벨이므로, 이 다음에 초기화할 하위 콤보박스가 없습니다.
    #     else:
    #         print(f"경고: 유효하지 않은 단지 선택: {selected_danji_name}. 면적 목록을 로드할 수 없습니다.")
    #         self.clear_naver_lower_combos(['myeoneok'])
# --- 삭제 끝 ---

    def load_naver_myeoneok_list(self, danji_code):
        """
        네이버 부동산 면적 목록을 API를 직접 호출하여 로드합니다.
        (Chrome 개발자 도구에서 추출한 상세 헤더 적용)
        """
        print(f"load_naver_myeoneok_list: 단지 코드 {danji_code}에 대한 면적 목록 (API 호출) 로드 시작.")
        
        # --- 수정 시작 (API URL 사용) ---
        url = f"https://fin.land.naver.com/front-api/v1/complex/pyeongList?complexNumber={danji_code}"
        # --- 수정 끝 ---
        
        print(f"load_naver_myeoneok_list - 요청 URL: {url}")
        
        # --- 수정 시작 (Selenium/BeautifulSoup 관련 코드 삭제, API 호출 로직으로 복구) ---
        # 딜레이 추가 (requests 방식에서도 429 오류가 발생할 수 있으므로 유지)
        time.sleep(1) 
        
        data = self.fetch_naver_api_data(url) 
        
        print(f"load_naver_myeoneok_list - fetch_naver_api_data 응답 데이터: {data}")

        self.naver_myeoneok_map = {}
        myeoneok_display_list = []

        if data and data.get('isSuccess') and 'result' in data and isinstance(data['result'], list):
            myeoneok_list_raw = data['result'] 
            
            print(f"load_naver_myeoneok_list - 필터링 전 면적 raw 리스트: {myeoneok_list_raw}")

            for item in myeoneok_list_raw:
                myeoneok_name = item.get('name')
                myeoneok_code = item.get('number') 
                exclusive_area = item.get('exclusiveArea')
                supply_area = item.get('supplyArea')
                
                if myeoneok_name and myeoneok_code is not None:
                    display_text = f"{myeoneok_name}"
                    if exclusive_area is not None:
                        display_text += f" (전용 {exclusive_area:.2f}㎡"
                        if supply_area is not None:
                            display_text += f" / 공급 {supply_area:.2f}㎡)"
                        else:
                            display_text += ")"
                    
                    self.naver_myeoneok_map[display_text] = myeoneok_code
                    myeoneok_display_list.append(display_text)
            
            print(f"load_naver_myeoneok_list - 최종 면적 목록: {myeoneok_display_list}")

            self.naver_myeoneok_combobox['values'] = myeoneok_display_list
            
            if myeoneok_display_list:
                self.naver_myeoneok_combobox.set(myeoneok_display_list[0])
            else:
                self.naver_myeoneok_combobox.set("면적 선택")
        else:
            print(f"네이버 면적 목록 로드 실패 (단지 코드: {danji_code}) - 응답 데이터 문제 또는 형식 불일치")
            self.naver_myeoneok_combobox['values'] = ["목록 로드 실패"]
            self.naver_myeoneok_combobox.set("목록 로드 실패")
        # --- 수정 끝 ---
        
        self.root.after(0, self.root.update_idletasks) # UI 갱신 강제

    def on_naver_myeoneok_selected(self, event):
        """네이버 면적 선택 시 동작 (현재는 매물 검색 버튼으로 이동)"""
        print("on_naver_myeoneok_selected - 면적 선택됨. 매물 검색 버튼을 눌러주세요.")

    def search_naver_properties(self):
        """네이버 매물 검색 버튼 클릭 시 동작 (매매/전세 동시 검색)"""
        selected_danji_name = self.naver_danji_combobox.get()
        selected_danji_code = self.naver_danji_map.get(selected_danji_name)
        selected_myeoneok_name = self.naver_myeoneok_combobox.get()
        selected_myeoneok_code = self.naver_myeoneok_map.get(selected_myeoneok_name) 

        if not selected_danji_code or not selected_myeoneok_code:
            messagebox.showwarning("경고", "단지와 면적을 선택해주세요.")
            self.loading_label.config(text="매물 검색 실패: 단지/면적 선택 필요")
            return

        self.loading_label.config(text="매물 데이터를 조회 중입니다...")
        
        # 매매, 전세 동시 검색을 위한 스레드 생성
        buy_thread = threading.Thread(target=self._fetch_and_update_property_list, 
                                     args=('A1', '매매', selected_danji_code, selected_myeoneok_code))
        rent_thread = threading.Thread(target=self._fetch_and_update_property_list, 
                                      args=('B1', '전세', selected_danji_code, selected_myeoneok_code))
        
        buy_thread.daemon = True
        rent_thread.daemon = True
        
        buy_thread.start()
        rent_thread.start()

        self.root.after(100, self._check_property_fetches_complete, buy_thread, rent_thread)


    def _fetch_and_update_property_list(self, trade_type_code, trade_type_name, danji_code, myeoneok_code):
        """매물 데이터를 가져와 특정 Treeview를 업데이트하는 내부 도우미 함수"""
        try:
            url = f"https://fin.land.naver.com/front-api/v1/complex/article/list?complexNumber={danji_code}&pyeongTypeNumbers%5B%5D={myeoneok_code}&tradeTypes%5B%5D={trade_type_code}&dateDescending=false&userChannelType=PC&page=0"
            data = self.fetch_naver_api_data(url)
            
            if data and 'result' in data:
                article_data = data['result']
                # Tkinter 메인 스레드에서 UI 업데이트 함수 호출
                self.root.after(0, self._update_single_property_list_ui, article_data, trade_type_name)
            else:
                self.root.after(0, self.show_property_search_error, f"{trade_type_name} 매물 데이터를 가져올 수 없습니다.")
                
        except Exception as e:
            error_msg = f"{trade_type_name} 매물 검색 중 오류 발생: {str(e)}"
            self.root.after(0, self.show_property_search_error, error_msg)

    def _check_property_fetches_complete(self, buy_thread, rent_thread):
        """매물 검색 스레드의 완료 여부를 주기적으로 확인"""
        if buy_thread.is_alive() or rent_thread.is_alive():
            self.root.after(100, self._check_property_fetches_complete, buy_thread, rent_thread)
        else:
            self.loading_label.config(text="매물 검색 완료.") # 모든 검색 완료 시 메시지 업데이트

    def _sort_naver_property_tree(self, tree_widget, col, trade_type):
        # 어떤 Treeview의 정렬 방향을 사용할지 결정
        if trade_type == '매매':
            reverse_sort = not self.buy_property_tree_sort_direction
            self.buy_property_tree_sort_direction = reverse_sort
        else: # '전세'
            reverse_sort = not self.rent_property_tree_sort_direction
            self.rent_property_tree_sort_direction = reverse_sort

        current_data = []
        for item_id in tree_widget.get_children():
            # Treeview 아이템의 값을 가져옵니다.
            value = tree_widget.set(item_id, col)
            current_data.append((value, item_id))

        def get_sort_key_for_price(item_tuple):
            price_text = item_tuple[0] # 예: "7.500 억원" 또는 "300/50 만원"
            
            # 숫자만 추출하여 float으로 변환
            try:
                # "억원" 단위 가격 처리
                if "억원" in price_text:
                    clean_price = price_text.replace(' 억원', '').strip()
                    return float(clean_price)
                # "만원" 단위 (매매/전세실거래 등) 또는 "보증금/월세 만원" 처리
                elif "만원" in price_text:
                    if '/' in price_text: # 월세 (보증금/월세 형태)는 복잡하므로 보증금으로 정렬
                        parts = price_text.split('/')
                        deposit = float(parts[0].replace('만원','').replace(',','').strip())
                        return deposit # 월세는 보증금 기준으로 정렬
                    else: # 일반 "만원" 단위 (예: 5000 만원)
                        clean_price = price_text.replace('만원','').replace(',','').strip()
                        return float(clean_price)
                else: # 가격 정보가 없거나, 숫자 변환 불가 ("-")
                    return -float('inf') # 정렬 시 가장 낮은 값으로 처리 (맨 위로)
            except ValueError:
                return -float('inf') # 변환 실패 시 (예외 발생) 가장 낮은 값으로 처리

        # '가격' 컬럼에 대해서만 get_sort_key_for_price 사용
        current_data.sort(key=get_sort_key_for_price, reverse=reverse_sort)

        # Treeview 아이템 재정렬
        for index, (value, item_id) in enumerate(current_data):
            tree_widget.move(item_id, '', index)

        # 컬럼 헤더 텍스트 업데이트 (정렬 방향 표시)
        for c in tree_widget['columns']:
            if c == col:
                tree_widget.heading(c, text=f"{c} {'▼' if reverse_sort else '▲'}")
            else:
                tree_widget.heading(c, text=c)

    def _update_single_property_list_ui(self, article_data, trade_type_name):
        """단일 매물 유형(매매/전세)의 Treeview와 요약 정보를 업데이트"""
        article_list = article_data.get('list', [])
        total_count = article_data.get('totalCount', 0)
        
        target_tree = self.buy_property_tree if trade_type_name == "매매" else self.rent_property_tree

        # 기존 목록 삭제
        for item in target_tree.get_children():
            target_tree.delete(item)
        
        # 요약 정보 계산을 위한 초기값
        min_price_val = float('inf') 
        max_price_val = 0.0 
        
        if not article_list:
            self.loading_label.config(text=f"{trade_type_name} 매물이 없습니다.")
            # 요약 레이블 업데이트
            trade_type_key = "buy" if trade_type_name == "매매" else "rent"
            self.naver_summary_labels[f'{trade_type_key}_count'].config(text=f"{trade_type_name} 매물건수: 0개")
            self.naver_summary_labels[f'{trade_type_key}_max_price'].config(text=f"{trade_type_name} 최고가: -억원")
            self.naver_summary_labels[f'{trade_type_key}_min_price'].config(text=f"{trade_type_name} 최저가: -억원")
            return
        
        # 매물 목록 추가 및 요약 정보 계산
        for item in article_list: # item은 각 매물(Article)의 정보를 담은 딕셔너리
            try:
                # rep_info 아래에 대부분의 매물 상세 정보가 있음
                rep_info = item.get('representativeArticleInfo', {})
                price_info = rep_info.get('priceInfo', {})
                # articleDetail은 rep_info 내에 articleDetail 딕셔너리가 또 중첩
                article_detail = rep_info.get('articleDetail', {}) 

                # 1. articleNo (매물 번호) 추출 로직 강화
                # 매물 번호는 'articleNo' 키로 rep_info 안에 있을 확률이 높습니다.
                # 혹시 rep_info 안에 없다면, item 바로 아래에 있을 수도 있습니다.
                article_no = rep_info.get('articleNumber') 
                if article_no is None: # rep_info 안에 없다면 item 바로 아래도 확인
                    article_no = item.get('articleNumber')
                
                # print(f"디버그: 추출된 articleNo (iid): {article_no}") # 추출된 값 확인
                
                if not article_no: # articleNo가 유효하지 않으면 이 매물은 건너뜁니다.
                    print(f"경고: articleNo를 찾을 수 없어 이 매물을 건너뜜. 원본 아이템: {item}")
                    continue 

                # 2. 층 정보 추출 로직 강화
                # 'floorInfo' 키가 없거나 값이 비어있는 경우를 대비하여 다른 가능한 키를 시도합니다.
                floor_info = article_detail.get('floorInfo', '-') 
                if floor_info == '-' or floor_info == '': # 'floorInfo'가 없거나 빈 문자열이면
                    floor_info = article_detail.get('floor', '-') # 'floor'라는 다른 키 시도
                if floor_info == '-' or floor_info == '': # 여전히 없으면 rep_info 바로 아래에 있는지 시도
                     floor_info = rep_info.get('floor', '-')

                # print(f"디버그: 추출된 층 정보: {floor_info}") # 추출된 값 확인

                # 3. 가격 정보 추출
                price_text = "-"
                price_for_summary = None 
                if trade_type_name == "매매":
                    raw_price_won = price_info.get('dealPrice', 0) 
                    if raw_price_won is not None and raw_price_won > 0:
                        price_in_eokwon = raw_price_won / 100000000.0 
                        price_text = f"{price_in_eokwon:.3f} 억원"
                        price_for_summary = price_in_eokwon
                elif trade_type_name == "전세":
                    raw_price_won = price_info.get('warrantyPrice', 0) 
                    if raw_price_won is not None and raw_price_won > 0:
                        price_in_eokwon = raw_price_won / 100000000.0 
                        price_text = f"{price_in_eokwon:.3f} 억원"
                        price_for_summary = price_in_eokwon
                    if price_info.get('rentPrice', 0) > 0: # 월세가 존재하는 경우
                        deposit_price = price_info.get('warrantyPrice', 0)
                        rent_price = price_info.get('rentPrice', 0)
                        price_text = f"{deposit_price:,}/{rent_price:,} 만원"
                        price_for_summary = None # 월세는 요약 통계 계산에서 제외

                # min_price_val, max_price_val 업데이트
                if price_for_summary is not None:
                    if price_for_summary < min_price_val:
                        min_price_val = price_for_summary
                    if price_for_summary > max_price_val:
                        max_price_val = price_for_summary

                # 4. 동 정보 추출
                dong_name = rep_info.get('dongName', '-')

                # 5. 면적 정보 추출
                # 면적 콤보박스의 현재 값에서 ' (타입)' 부분을 제거하여 사용
                full_area_text_from_combo = self.naver_myeoneok_combobox.get()
                if ' (' in full_area_text_from_combo:
                    area_text = full_area_text_from_combo.split(' (')[0]
                else:
                    area_text = full_area_text_from_combo

                # Treeview에 추가: article_no를 iid로 사용하여 웹페이지 연결의 근거로 삼음
                target_tree.insert('', tk.END, values=(price_text, dong_name, floor_info, area_text), iid=str(article_no))
                
            except Exception as e:
                # 파싱 중 오류 발생 시 어떤 매물에서 어떤 오류가 발생했는지 콘솔에 출력
                print(f"매물 정보 파싱 중 오류 (유형: {trade_type_name}): {e}, 원본 아이템: {item}")
                continue
        
        # 매물 현황 요약 레이블 업데이트
        trade_type_key = "buy" if trade_type_name == "매매" else "rent"
        self.naver_summary_labels[f'{trade_type_key}_count'].config(text=f"{trade_type_name} 매물건수: {total_count}개")
        
        # 가격 요약 레이블은 min_price_val/max_price_val이 업데이트 된 경우에만 숫자로 표시
        self.naver_summary_labels[f'{trade_type_key}_max_price'].config(text=f"{trade_type_name} 최고가: {max_price_val:.3f}억원" if max_price_val != 0.0 else f"{trade_type_name} 최고가: -억원")
        self.naver_summary_labels[f'{trade_type_key}_min_price'].config(text=f"{trade_type_name} 최저가: {min_price_val:.3f}억원" if min_price_val != float('inf') else f"{trade_type_name} 최저가: -억원")
        # debug_label은 _create_naver_property_search_에 정의되어야 합니다.

    def _open_naver_article_webpage(self, event):
        """매물 목록 Treeview 항목을 더블클릭했을 때 네이버 부동산 웹페이지를 엽니다."""
        tree_widget = event.widget 
        selected_item_id = tree_widget.focus() # 더블클릭된 항목의 iid (articleNo가 될 것)

        print(f"더블클릭 이벤트 발생. selected_item_id: {selected_item_id}") # 디버깅 출력

        if selected_item_id:
            article_no = selected_item_id # Treeview의 iid가 articleNo로 넘어와야 함

            # 'I001'과 같은 잘못된 값이 아닌, 실제 유효한 매물 번호인지 확인
            if article_no and str(article_no).isdigit(): # article_no가 비어있지 않고, 숫자로만 구성되어 있는지 확인
                # 네이버 매물 상세 페이지 URL (현재는 fin.land.naver.com 도메인을 사용)
                article_url = f"https://fin.land.naver.com/articles/{article_no}"
                
                try:
                    webbrowser.open(article_url)
                    print(f"웹페이지 열기 시도: {article_url}")
                    self.update_debug_label(f"매물 웹페이지 열림: {article_url}")
                except Exception as e:
                    messagebox.showerror("웹페이지 열기 오류", f"웹 브라우저를 열 수 없습니다: {e}")
                    self.update_debug_label(f"웹페이지 열기 오류: {e}")
            else:
                messagebox.showwarning("정보 없음", f"선택된 매물 항목의 웹페이지 정보(매물 번호)가 유효하지 않습니다: '{article_no}'")
                self.update_debug_label(f"유효하지 않은 매물 번호: {article_no}")
        else:
            print("선택된 매물 항목이 없습니다.")
            self.update_debug_label("매물 항목 미선택.")

    def update_debug_label(self, message):
        """디버깅 라벨 텍스트를 업데이트하는 메서드""" # <-- def 라인보다 정확히 4칸 들여쓰기 되어야 합니다.
        if self.debug_label:
           self.debug_label.config(text=message)
           self.root.update_idletasks()

    def clear_naver_lower_combos(self, combo_types):
        """네이버 매물 검색창 하위 콤보박스들을 초기화합니다."""
        # --- 삭제 시작 (시군구 관련 초기화 - 기존 삭제됨) ---
        # if 'sigungu' in combo_types:
        #     self.naver_sigungu_combobox.set("시군구 선택")
        #     self.naver_sigungu_combobox['values'] = []
        #     self.naver_sigungu_map = {}
        # --- 삭제 끝 ---

        # --- 삭제 시작 (법정동 관련 초기화 - 기존 삭제됨) ---
        # if 'bjdong' in combo_types:
        #     self.naver_bjdong_combobox.set("법정동 선택")
        #     self.naver_bjdong_combobox['values'] = []
        #     self.naver_bjdong_map = {}
        # --- 삭제 끝 ---

        # --- 수정 시작 (단지 관련 초기화 및 상태 변경) ---
        if 'danji' in combo_types:
            self.naver_danji_combobox.set("단지 선택")
            self.naver_danji_combobox['values'] = []
            self.naver_danji_map = {}
            self.naver_danji_combobox.config(state="normal") # 초기화 시 다시 활성화 (새 값 받을 준비)
        # --- 수정 끝 ---

        # --- 유지 (면적 관련 초기화) ---
        if 'myeoneok' in combo_types:
            self.naver_myeoneok_combobox.set("면적 선택")
            self.naver_myeoneok_combobox['values'] = []
            self.naver_myeoneok_map = {}
        # --- 유지 끝 ---

        self.root.update_idletasks() # UI 갱신 강제

    # ==============================================================================
    # 5. 메인 실행 메서드 (가장 마지막에 배치)
    # ==============================================================================

    def run(self):
        """Tkinter 메인 루프 실행"""
        self.root.mainloop()

if __name__ == "__main__":
    # --- 수정 시작: 앱 객체 생성 시 만료일 문자열을 전달 ---
    app = InvestmentTableProgram(PROGRAM_EXPIRATION_DATE_STR)
    # --- 수정 끝 ---
    app.run()

pywin32 라이브러리를 성공적으로 로드했습니다.
DPI 인식 설정을 시도했습니다.
[GitHub]에서 지역코드 파일 로드 시도: https://raw.githubusercontent.com/kaga-fo/DaonSearcher_Project/refs/heads/main/%EC%A7%80%EC%97%AD%EC%BD%94%EB%93%9C.txt
[GitHub]에서 지역코드 파일 성공적으로 로드.
지역코드 파싱 완료. 총 17개의 시도 데이터.
load_naver_myeoneok_list: 단지 코드 142919에 대한 면적 목록 (API 호출) 로드 시작.
load_naver_myeoneok_list - 요청 URL: https://fin.land.naver.com/front-api/v1/complex/pyeongList?complexNumber=142919
API 요청 오류: 429 Client Error: Too Many Requests for url: https://fin.land.naver.com/front-api/v1/complex/pyeongList?complexNumber=142919
응답 코드: 429
{"detailCode":"TOO_MANY_REQUESTS","message":""}
load_naver_myeoneok_list - fetch_naver_api_data 응답 데이터: None
네이버 면적 목록 로드 실패 (단지 코드: 142919) - 응답 데이터 문제 또는 형식 불일치
